<a href="https://colab.research.google.com/github/globalenglish01/Billion/blob/main/billion_runall_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [206]:
import importlib.util
import sys
import subprocess

def install_package(package_name):
    if importlib.util.find_spec(package_name) is None:
        print(f"{package_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
    else:
        print(f"{package_name} is already installed.")

# 示例：检查并安装 pandas 和 numpy
#install_package("pandas")
#install_package("numpy")

def install_packages(package_list):
    for package in package_list:
        install_package(package)

install_packages(["gTTS", "pypinyin", "requests", "pandas", "rich", "akshare"])

gTTS not found. Installing...
pypinyin is already installed.
requests is already installed.
pandas is already installed.
rich is already installed.
akshare is already installed.


In [207]:
from datetime import datetime
from zoneinfo import ZoneInfo

def get_beijing_time():
    # 获取当前 UTC 时间
    utc_now = datetime.now(ZoneInfo("UTC"))
    # 转换为北京时间
    beijing_now = utc_now.astimezone(ZoneInfo("Asia/Shanghai"))
    # 格式化日期为字符串
    formatted_date = beijing_now.strftime("%Y-%m-%d")
    return formatted_date

# 示例调用
print(get_beijing_time())

2025-06-12


In [208]:

#install_packages(["gTTS", "pypinyin"])

from gtts import gTTS
from IPython.display import Audio, display
import os
import pypinyin  # 用于拼音转换
import time
from IPython.core.display import display, HTML

def generate_and_play_stock_sounds_fromText(code, text):
    """
    针对多个股票生成详细语音并依次播放。
    :param stock_data: 股票数据列表，例如 [{"code": "000001", "name": "平安银行", "price": 12.34, "change": "+3.2%"}]
    """
    # 临时文件存储目录
    temp_dir = "stock_sounds"
    os.makedirs(temp_dir, exist_ok=True)

    # 遍历股票数据生成语音
    """
    for stock in stock_data:
        # 拼音转换
        #code_pinyin = " ".join(pypinyin.lazy_pinyin(stock['stockID']))
        name_pinyin = " ".join(pypinyin.lazy_pinyin(stock['stockName']))

        # 构造语音播报内容，拼音化股票代码
        #text = (
        #    f"股票代码 {code_pinyin}，"
        text = (
            f"股票代码 {str(stock['stockID'])}，"
            f"名称 {name_pinyin}，"
            f"当前价格 {stock['stockPrice']} 元，"
            f"涨跌幅 {stock['changesPercent']}。"
        )
    """

    # 使用 gTTS 生成语音
    tts = gTTS(text=text, lang="zh-cn")

    time.sleep(1)

    # 保存为临时文件
    temp_file = os.path.join(temp_dir, f"{code}.mp3")
    tts.save(temp_file)

    # 播放语音
    #print("\033[1;31;43m" + text + "\033[0m")
    #print(f"正在播报:\033[1;31;43m {text}\033[0m")
    #print(f"正在播报:\033[1;31m {text}\033[0m")
    #print(f"正在播报:\033[1;91m {text}\033[0m")
    #print(f"正在播报:[bold bright_red]{text}[/]")

    display(HTML(f'<span style="font-weight:bold; color:#FF0000;">{text}</span>'))

    display(Audio(temp_file, autoplay=True))
    #Audio(temp_file, autoplay=True)

#print(f"stock_id👍：股票 {StockName}（{stock_id}）: 在 {time} 开盘后最高价更新为 {price}（涨幅：{percent:.2f}%），{stock_data['base_high_updates']} 次创新高，买入信号触发")

In [209]:
from datetime import datetime

def seconds_between(time1: str, time2: str) -> int:
    # 解析字符串为时间对象
    t1 = datetime.strptime(time1, "%H%M%S")
    t2 = datetime.strptime(time2, "%H%M%S")

    # 计算时间差（秒数）
    return abs((t2 - t1).seconds)

# 示例
#time1 = "94600"
#time2 = "94627"
#print(seconds_between(time1, time2))  # 输出 604

In [210]:
def getMarketID(stock_id):
    """根据股票代码获取市场 ID（深市或沪市）"""
    if str(stock_id).zfill(6) < '600000':
        return '0'  # 深市
    else:
        return '1'  # 沪市

def getMarketID2(stock_id):
    """根据股票代码获取市场 ID 备用（例如：用于第二次使用的市场类型）"""
    if str(stock_id).zfill(6) < '600000':
        return '0'  # 深市
    else:
        return '1'  # 沪市

In [211]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_limit_up_stats_ndays_old(stock_code, is_st=False, ndays=180):
    """
    获取指定股票最近N个交易日的涨停个数和涨停率

    :param stock_code: 股票代码（如 "300750"）
    :param is_st: 是否ST股（决定涨停为5%或20%）
    :param ndays: 统计的交易日数量（默认120天）
    :return: dict，包含涨停个数和涨停率
    """
    # 获取当前日期
    today = datetime.now().strftime("%Y-%m-%d")

    # 判断市场
    secid = f"0.{stock_code}" if (stock_code.startswith("3") or stock_code.startswith("0")) else f"1.{stock_code}"

    url = "http://push2his.eastmoney.com/api/qt/stock/kline/get"
    params = {
        "secid": secid,
        "fields1": "f1,f2,f3,f4,f5,f6",
        "fields2": "f51,f52,f53,f54,f55,f56,f57",
        "klt": "101",  # 日K线
        "fqt": "1",    # 前复权
        "end": "20500101",
        "lmt": 300     # 提前多取，后面再截取最近N天
    }

    headers = {"User-Agent": "Mozilla/5.0"}
    #r = requests.get(url, params=params, headers=headers)
    #data = r.json()
    try:
        r = requests.get(url, params=params, headers=headers, timeout=10)
        #r.raise_for_status()  # 检查 HTTP 状态码

        data = r.json() # 修正 JSON 解析
    except requests.exceptions.Timeout:
        print("请求超时，请稍后重试...")
        data = "" # 修正 JSON 解析
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        data = "" # 修正 JSON 解析


    if "data" not in data or "klines" not in data["data"]:
        return {
        "stock_code": stock_code,
        "stock_name": "无法获取K线数据",
        "days": ndays,
        "total_days": 0,
        "limit_up_days": 0,
        "limit_up_ratio": 0.00
        }
        #return {"error": "无法获取K线数据"}
    stock_name = data["data"]["name"]

    raw = data["data"]["klines"]
    df = pd.DataFrame([i.split(",") for i in raw],
                      columns=["date", "open", "close", "high", "low", "volume", "amount"])
    df = df.astype({"open": float, "close": float})
    df["prev_close"] = df["close"].shift(1)
    df["date"] = pd.to_datetime(df["date"])

    # 剔除今天的数据（如果今天有的话）
    df = df[df["date"] < pd.to_datetime(today)]

    # 取最近 N 天
    df = df.dropna().sort_values("date").tail(ndays).copy()

    if df.empty:
        return {
        "stock_code": stock_code,
        "stock_name": "无足够数据",
        "days": ndays,
        "total_days": 0,
        "limit_up_days": 0,
        "limit_up_ratio": 0.00
        }
        #return {"error": "无足够数据"}

    # 涨停价 = 昨收 * (1.20 or 1.05)
    #limit_ratio = 1.05 if is_st else 1.20
    limit_ratio = 1.20
    df["limit_price"] = (df["prev_close"] * limit_ratio).round(2)
    df["is_limit_up"] = (df["close"] >= df["limit_price"]) & (df["amount"].astype(float) > 0)

    total_days = df["prev_close"].notna().sum()
    limit_up_days = df["is_limit_up"].sum()
    limit_up_ratio = round(limit_up_days / total_days * 100, 2) if total_days > 0 else 0


    print(f"股票代码: {stock_code}")
    print(f"股票名称: {stock_name}")
    print(f"统计周期: 最近 {ndays} 个交易日")
    print(f"有效交易日数: {total_days}")
    if int(limit_up_days) >= 1:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">涨停次数: {limit_up_days} 次</span>'))
    else:
        print(f"涨停次数: {limit_up_days} 次")
    print(f"涨停率: {limit_up_ratio}%")

    if limit_up_ratio > 10:
        print("这个股票涨停比较多，值得关注")

    return {
        "stock_code": stock_code,
        "stock_name": stock_name,
        "days": ndays,
        "total_days": total_days,
        "limit_up_days": int(limit_up_days),
        "limit_up_ratio": limit_up_ratio
    }

# 查询股票最近120个交易日的涨停情况（创业板，非ST）
#result = get_limit_up_stats_ndays("300661", is_st=False, ndays=120)
result = get_limit_up_stats_ndays_old("301665")
print(result)

股票代码: 301665
股票名称: 泰禾股份
统计周期: 最近 180 个交易日
有效交易日数: 39
涨停次数: 0 次
涨停率: 0.0%
{'stock_code': '301665', 'stock_name': '泰禾股份', 'days': 180, 'total_days': np.int64(39), 'limit_up_days': 0, 'limit_up_ratio': np.float64(0.0)}


In [212]:
import requests
import pandas as pd
from datetime import datetime
from IPython.core.display import display, HTML

def get_limit_up_stats_ndays(stock_code, is_st=False, ndays=180):
    """
    获取指定股票最近N个交易日的涨停个数和涨停率，以及最近两天是否涨停
    """
    today = datetime.now().strftime("%Y-%m-%d")

    # 判断市场代码
    secid = f"0.{stock_code}" if (stock_code.startswith("3") or stock_code.startswith("0")) else f"1.{stock_code}"

    url = "http://push2his.eastmoney.com/api/qt/stock/kline/get"
    params = {
        "secid": secid,
        "fields1": "f1,f2,f3,f4,f5,f6",
        "fields2": "f51,f52,f53,f54,f55,f56,f57",
        "klt": "101",
        "fqt": "1",
        "end": "20500101",
        "lmt": 300
    }

    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        r = requests.get(url, params=params, headers=headers, timeout=10)
        data = r.json()
    except requests.exceptions.Timeout:
        print("请求超时，请稍后重试...")
        return {}
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return {}

    if "data" not in data or "klines" not in data["data"]:
        return {
            "stock_code": stock_code,
            "stock_name": "无法获取K线数据",
            "days": ndays,
            "total_days": 0,
            "limit_up_days": 0,
            "limit_up_ratio": 0.00,
            "recent_two_days_info": []
        }

    stock_name = data["data"]["name"]
    raw = data["data"]["klines"]
    df = pd.DataFrame([i.split(",") for i in raw],
                      columns=["date", "open", "close", "high", "low", "volume", "amount"])
    df = df.astype({"open": float, "close": float})
    df["prev_close"] = df["close"].shift(1)
    df["date"] = pd.to_datetime(df["date"])

    df = df[df["date"] < pd.to_datetime(today)]
    df = df.dropna().sort_values("date").tail(ndays).copy()

    if df.empty:
        return {
            "stock_code": stock_code,
            "stock_name": "无足够数据",
            "days": ndays,
            "total_days": 0,
            "limit_up_days": 0,
            "limit_up_ratio": 0.00,
            "recent_two_days_info": []
        }

    limit_ratio = 1.20 if is_st else 1.20
    df["limit_price"] = (df["prev_close"] * limit_ratio).round(2)
    df["is_limit_up"] = (df["close"] >= df["limit_price"]) & (df["amount"].astype(float) > 0)

    total_days = df["prev_close"].notna().sum()
    limit_up_days = df["is_limit_up"].sum()
    limit_up_ratio = round(limit_up_days / total_days * 100, 2) if total_days > 0 else 0

    print(f"股票代码: {stock_code}")
    print(f"股票名称: {stock_name}")
    print(f"统计周期: 最近 {ndays} 个交易日")
    print(f"有效交易日数: {total_days}")
    if int(limit_up_days) >= 1:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">涨停次数: {limit_up_days} 次</span>'))
    else:
        print(f"涨停次数: {limit_up_days} 次")
    print(f"涨停率: {limit_up_ratio}%")

    if limit_up_ratio > 10:
        print("这个股票涨停比较多，值得关注")

    # 获取最近两个交易日的涨停信息
    recent_two = df.sort_values("date").tail(2)
    recent_two_days_info = recent_two[["date", "is_limit_up"]].copy()
    recent_two_days_info["date"] = recent_two_days_info["date"].dt.strftime("%Y-%m-%d")
    recent_two_days_info = recent_two_days_info.to_dict(orient="records")


    # 倒数第二天
    if recent_two_days_info[0]['is_limit_up']:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">倒数第二天 ({recent_two_days_info[0]["date"]}) 是否涨停: {recent_two_days_info[0]["is_limit_up"]}</span>'))
    else:
        print(f"倒数第二天 ({recent_two_days_info[0]['date']}) 是否涨停: {recent_two_days_info[0]['is_limit_up']}")

    # 最后一天
    if recent_two_days_info[1]['is_limit_up']:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">倒数第一天 ({recent_two_days_info[1]["date"]}) 是否涨停: {recent_two_days_info[1]["is_limit_up"]}</span>'))
    else:
        print(f"倒数第一天 ({recent_two_days_info[1]['date']}) 是否涨停: {recent_two_days_info[1]['is_limit_up']}")
    #print(f"倒数第一天({recent_two_days_info[1]['date']})是否涨停: {recent_two_days_info[1]['is_limit_up']}")

    return {
        "stock_code": stock_code,
        "stock_name": stock_name,
        "days": ndays,
        "total_days": total_days,
        "limit_up_days": int(limit_up_days),
        "limit_up_ratio": limit_up_ratio,
        "recent_two_days_info": recent_two_days_info
    }

# 示例调用
result = get_limit_up_stats_ndays("301112")
print(result)

股票代码: 301112
股票名称: 信邦智能
统计周期: 最近 180 个交易日
有效交易日数: 180


涨停率: 1.67%
倒数第二天 (2025-06-10) 是否涨停: False
倒数第一天 (2025-06-11) 是否涨停: False
{'stock_code': '301112', 'stock_name': '信邦智能', 'days': 180, 'total_days': np.int64(180), 'limit_up_days': 3, 'limit_up_ratio': np.float64(1.67), 'recent_two_days_info': [{'date': '2025-06-10', 'is_limit_up': False}, {'date': '2025-06-11', 'is_limit_up': False}]}


In [213]:

#!pip install akshare
import akshare as ak

def get_stock_popularity_rank(stock_code: str) -> int:
    """
    获取指定股票在东方财富网个股人气榜中的实时排名
    :param stock_code: 股票代码，例如 '000001'
    :return: 排名（整数），如果未找到则返回 None
    """
    # 确定股票市场前缀
    if stock_code.startswith(('6')):  # 上海证券交易所股票代码以6或9开头
        symbol = f"SH{stock_code}"
    else:  # 深圳证券交易所股票代码以0、2、3开头
        symbol = f"SZ{stock_code}"

    # 获取个股人气榜实时数据
    try:
        data = ak.stock_hot_rank_detail_realtime_em(symbol=symbol)
        if not data.empty:
            # 获取最新一条数据的排名
            latest_rank = int(data.iloc[-1]['排名'])
            if latest_rank <= 100:
                display(HTML(f'<span style="font-weight:bold; color:#FF0000;">股票代码 {stock_code} 当前在个股人气榜中的排名为: {latest_rank}</span>'))
            else:
                print(f"股票代码 {stock_code} 当前在个股人气榜中的排名为: {latest_rank}")

            return latest_rank
        else:
            print(f"未找到股票代码 {stock_code} 的人气榜数据。")
            return None
    except Exception as e:
        print(f"获取数据时出现错误: {e}")
        return None

# 示例调用
stock_code = '301665'  # 替换为您感兴趣的股票代码
rank = get_stock_popularity_rank(stock_code)

股票代码 301665 当前在个股人气榜中的排名为: 1002


In [214]:

import requests
import json
import re
import pandas as pd

# **1. 解析 JSONP 格式**
def parse_jsonp(jsonp_str):
    """解析 JSONP 数据，提取 JSON 并解析"""
    data = jsonp_str
    # 解析返回的 JSON 或 HTML 数据
    if 'jQuery' in data:
        # 去掉回调函数包裹的部分，只提取 JSON 数据
        data = data[data.index('(')+1:data.rindex(')')]
        try:
            json_data = eval(data)  # 这里假设返回的就是一个字典类型的数据
            return json_data
        except Exception as e:
            print(f"解析数据时出错：{e}")
            #raise ValueError(f"Invalid JSONP format: {jsonp_str[:500000]}")  # 打印前 500 个字符，帮助调试
            #break
            return ""
    else:
        print("返回数据格式不符合预期")
        #raise ValueError(f"Invalid JSONP format: {jsonp_str[:500000]}")  # 打印前 500 个字符，帮助调试
        #break
        return ""

#**2. 获取 tick 数据（每笔成交记录）**
def get_tick_data(stock_id):
    """
    获取每笔成交记录数据
    :param stock_id: 股票代码
    :return: 每笔成交数据列表
    """
    #url = f"http://push2ex.eastmoney.com/getStockFenShi?pagesize=144000&ut=7eea3edcaed734bea9cbfc24409ed989&dpt=wzfscj&pageindex=0&id={stock_id}1&sort=1&ft=1&code={stock_id}&market=1"
    url = "http://push2ex.eastmoney.com/getStockFenShi?pagesize=144000&ut=7eea3edcaed734bea9cbfc24409ed989&dpt=wzfscj&cb=jQuery1124019826114890089808_1611886439989&pageindex=0&id=" + '{:0>6}'.format(int(stock_id)) + getMarketID(int(stock_id)) + "&sort=1&ft=1&code=" + '{:0>6}'.format(int(stock_id)) + "&market=" + getMarketID2(int(stock_id)) + "&_=1611886439991"
    response = requests.get(url,timeout=10)
    data = parse_jsonp(response.text)  # 修正 JSON 解析
    return data['data']['data'] if 'data' in data and 'data' in data['data'] else []

# **3. 获取 minute 数据（每分钟净买额）**
def get_minute_data(stock_id):
    """
    获取每分钟净买额数据
    :param stock_id: 股票代码
    :return: 每分钟净买额数据列表
    """
    #url = f"http://push2.eastmoney.com/api/qt/stock/fflow/kline/get?cb=jQuery112305517775202340915_1612147619850&lmt=0&klt=1&fields1=f1%2Cf2%2Cf3%2Cf7&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61%2Cf62%2Cf63%2Cf64%2Cf65&ut=b2884a393a59ad64002292a3e90d46a5&secid=0.{stock_id}&_=1612147619878"
    url = "http://push2.eastmoney.com/api/qt/stock/fflow/kline/get?cb=jQuery112305517775202340915_1612147619850&lmt=0&klt=1&fields1=f1%2Cf2%2Cf3%2Cf7&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61%2Cf62%2Cf63%2Cf64%2Cf65&ut=b2884a393a59ad64002292a3e90d46a5&secid="+ getMarketID(int(stock_id)) + "." + '{:0>6}'.format(int(stock_id)) + "&_=1612147619878"
    #response = requests.get(url)

    try:
        response = requests.get(url, timeout=10)  # 设置超时时间为 10 秒
        #response.raise_for_status()  # 检查 HTTP 状态码

        data = parse_jsonp(response.text)  # 修正 JSON 解析
        return data['data']['klines'] if 'data' in data and 'klines' in data['data'] else []

    except requests.exceptions.Timeout:
        print("请求超时，请稍后重试...")
        return []
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return []

# **4. 解析每分钟净买额数据**
def parse_minute_data(minute_data):
    """
    解析 minute 数据，将字符串转换为字典
    :param minute_data: 每分钟净买额数据列表
    :return: 以时间为键的字典
    """
    minute_dict = {}
    for entry in minute_data:
        parts = entry.split(",")
        time = (int(parts[0].split(" ")[1].replace(":", "")) - 1) # 解析时间 (09:31 -> 931)
        net_amount = float(parts[1]) / 10000 # 净买额
        #net_small = float(parts[1])  # 净小单
        #net_medium = float(parts[2])  # 净中单
        #net_large = float(parts[3])  # 净大单
        #net_super = float(parts[4])  # 净超大单
        #net_superlarge = (net_large + net_super)/10000 # 净买额
        #minute_dict[time] = (net_small, net_medium, net_large, net_super, net_superlarge)
        minute_dict[time] = (net_amount)
    return minute_dict

# **5. 计算每笔成交涨幅，并匹配净买额**
def match_tick_with_minute(tick_data, minute_data, yesterday_close):
    """
    将每笔成交记录匹配对应的分钟净买额，并计算涨幅
    :param tick_data: 每笔成交记录
    :param minute_data: 每分钟净买额数据字典
    :param yesterday_close: 昨日收盘价
    :return: 处理后的 DataFrame
    """
    matched_data = []

    for tick in tick_data:
        minute = tick["t"] // 100 # 91525 -> 9:31:25 对应 931
        tick_time = tick["t"]
        price = tick["p"] / 1000  # 价格单位调整
        volume = tick["v"]  # 成交量
        direction = tick["bs"]  # 买卖方向
        change_percent = ((price - yesterday_close) / yesterday_close) * 100  # 计算涨幅

        # 获取该分钟的净买额数据
        if minute in minute_data:
            #net_small, net_medium, net_large, net_super, net_superlarge = minute_data[tick_time]
            net_amount = minute_data[minute]
        else:
            #net_small, net_medium, net_large, net_super, net_superlarge = (None, None, None, None, None)
            net_amount = (0)

        matched_data.append({
            "minute": minute,
            "time": tick_time,
            "price": price,
            "volume": volume,
            "direction": direction,
            "change_percent": change_percent,
            #"net_small": net_small,
            #"net_medium": net_medium,
            #"net_large": net_large,
            #"net_super": net_super,
            "net_amount": net_amount,
        })

    return pd.DataFrame(matched_data)

# **6. 执行完整流程**
#if __name__ == "__main__":
#def getNetAmoutByMin(stock_id,  yesterday_close):
def getNetAmoutByMin(stock_id):
    #stock_id = "301082"

    # **获取数据**
    #tick_data = get_tick_data(stock_id)
    minute_data = get_minute_data(stock_id)

    # **假设昨日收盘价**
    #yesterday_close = 14.52  # 需要根据实际情况调整

    # **解析 minute 数据**
    minute_dict = parse_minute_data(minute_data)

    # **匹配数据并计算涨幅**
    #result_df = match_tick_with_minute(tick_data, minute_dict, yesterday_close)

    # **显示前 10 行结果**
    #print(result_df.head(100))

    #return result_df
    return minute_dict

    #return (result_df[result_df["time"] == time])
    #stock_id = "301082"

    # **获取数据**
def getNetAmoutByTime(tick_with_minute, time):
    minute = time // 100 + 1 # 91525 -> 9:31:25 对应 931
    print(minute)
    #return ((tick_with_minute[tick_with_minute["time"] == minute])["net_amount"]).iloc[0]
    return tick_with_minute[minute]
"""
stock_id = "300180"
NetAmoutByMin = getNetAmoutByMin(stock_id)
print(NetAmoutByMin)
time = 134445
print(getNetAmoutByTime(NetAmoutByMin, time))
"""

'\nstock_id = "300180"\nNetAmoutByMin = getNetAmoutByMin(stock_id)\nprint(NetAmoutByMin)\ntime = 134445\nprint(getNetAmoutByTime(NetAmoutByMin, time))\n'

In [215]:
def get_nearest_smaller_value(data_dict, key):
    if key in data_dict:
        return data_dict[key]
    else:
        smaller_keys = [k for k in data_dict.keys() if k < key]
        if smaller_keys:
            nearest_key = max(smaller_keys)  # 选择比 key 小的最大值
            return data_dict[nearest_key]
        else:
            return None  # 如果没有比 key 小的值，返回 None

# 示例字典
data = {930: 100, 931: 200, 933: 300, 935: 400}

# 测试
print(get_nearest_smaller_value(data, 932))  # 输出 200（选择 931 的值）
print(get_nearest_smaller_value(data, 930))  # 输出 100（930 存在）
print(get_nearest_smaller_value(data, 929))  # 输出 None（没有比 929 小的键）

200
100
None


In [216]:
import bisect

def get_nearest_value(data_dict, key):
    if key in data_dict:
        return data_dict[key]  # 直接返回匹配的 key

    sorted_keys = sorted(data_dict.keys())  # 获取有序键列表
    index = bisect.bisect_right(sorted_keys, key)  # 找到 key 插入位置（右侧）

    # 尝试获取比 key 大的最近值
    if index < len(sorted_keys):
        return data_dict[sorted_keys[index]]

    # 尝试获取比 key 小的最近值
    if index > 0:
        return data_dict[sorted_keys[index - 1]]

    return 0  # 如果字典为空，则返回 None

# 示例数据
data = {930: 100, 931: 200, 933: 300, 935: 400}

# 测试
print(get_nearest_value(data, 932))  # 输出 300（优先取 933 的值）
print(get_nearest_value(data, 936))  # 输出 400（优先取 935 的值）
print(get_nearest_value(data, 929))  # 输出 100（优先取 930 的值）
print(get_nearest_value(data, 940))  # 输出 400（选 935 的值）
print(get_nearest_value(data, 930))  # 输出 100（key 存在）

300
400
100
400
100


In [217]:
def should_trigger_alert(sound_count, is_new_stock, NetBuyAmount, PanQianZTSec, is_high_now, is_high_open, stock_data):
    if sound_count != 0:
        return False

    condition1 = is_new_stock or ((NetBuyAmount >= 0) and (PanQianZTSec > 240) and is_high_now)

    condition2 = (
        (stock_data["low_updates"] == 0 and
         (stock_data["high_updates"] == 2 or (is_high_open and stock_data["high_updates"] == 1)))
        or
        (NetBuyAmount >= 5000 and stock_data["low_updates"] >= 1 and stock_data["high_updates"] == 1)
    )

    return condition1 and condition2

In [218]:




#install_packages(["requests", "pandas"])

import requests
import csv
from datetime import datetime

STOCK_TRACKER = {}


def get_all_fenshi_data_with_auctionOld(stock_id, date):
    """获取所有股票的分时数据，包括集合竞价数据（分页获取）
    https://16.push2delay.eastmoney.com/api/qt/stock/details/sse?fields1=f1,f2,f3,f4&fields2=f51,f52,f53,f54,f55&mpi=2000&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&pos=-0&secid=0.300846
    """
    base_url = "http://push2ex.eastmoney.com/getStockFenShi"
    page_size = 2000  # 设置每次请求的数据量
    ut = "7eea3edcaed734bea9cbfc24409ed989"
    dpt = "wzfscj"
    sort = 1
    ft = 1
    iscca = 1  # 设置为 1，获取集合竞价数据

    # 使用股票代码生成市场 ID
    market_id = getMarketID(stock_id)
    market_id2 = getMarketID2(stock_id)

    # 初始化第一页的 pageindex
    pageindex = 0
    all_data = []

    while True:
        # 构建 URL 请求
        url = f"{base_url}?pagesize={page_size}&ut={ut}&dpt={dpt}&cb=jQuery1124019826114890089808_1611886439989&pageindex={pageindex}&id={str(stock_id).zfill(6)}{market_id}&sort={sort}&ft={ft}&iscca={iscca}&code={str(stock_id).zfill(6)}&market={market_id2}&date={date}&_={1611886439991}"
        #print(url)
        # 发送请求并获取响应
        #response = requests.get(url, cookies=cookies)
        response = requests.get(url,timeout=10)
        if response.status_code == 200:
            data = response.text
            # 解析返回的 JSON 或 HTML 数据
            if 'jQuery' in data:
                # 去掉回调函数包裹的部分，只提取 JSON 数据
                data = data[data.index('(')+1:data.rindex(')')]
                try:
                    json_data = eval(data)  # 这里假设返回的就是一个字典类型的数据
                    all_data.extend(json_data['data']['data'])
                except Exception as e:
                    print(f"解析数据时出错：{e}")
                    all_data = []
                    break
            else:
                print("返回数据格式不符合预期")
                all_data = []
                break

            # 检查是否还有更多页的数据
            if len(json_data['data']['data']) < page_size:
                break  # 如果当前页的数据量少于设置的页面大小，说明是最后一页
            else:
                pageindex += 1  # 否则继续请求下一页
        else:
            print("请求失败，状态码:", response.status_code)
            all_data = []
            break

    return all_data
import requests
import json
import time

def get_all_fenshi_data_with_auction(stock_id, date):
    base_url = "http://push2ex.eastmoney.com/getStockFenShi"
    page_size = 2000
    ut = "7eea3edcaed734bea9cbfc24409ed989"
    dpt = "wzfscj"
    sort = 1
    ft = 1
    iscca = 1

    market_id = getMarketID(stock_id)
    market_id2 = getMarketID2(stock_id)

    pageindex = 0
    all_data = []

    while True:
        url = f"{base_url}?pagesize={page_size}&ut={ut}&dpt={dpt}&cb=jQuery1124019826114890089808_1611886439989&pageindex={pageindex}&id={str(stock_id).zfill(6)}{market_id}&sort={sort}&ft={ft}&iscca={iscca}&code={str(stock_id).zfill(6)}&market={market_id2}&date={date}&_={1611886439991}"

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"请求出错：{e}")
            time.sleep(1)  # 可选：稍等再试下一只股票
            break

        data = response.text
        if 'jQuery' in data:
            try:
                data = data[data.index('(')+1:data.rindex(')')]
                json_data = json.loads(data)
                all_data.extend(json_data['data']['data'])
            except Exception as e:
                print(f"解析数据时出错：{e}")
                break
        else:
            print("返回数据格式不符合预期")
            break

        if len(json_data['data']['data']) < page_size:
            break
        else:
            pageindex += 1

    return all_data
def filter_auction_data(all_data):
    """筛选出集合竞价期间的数据"""
    auction_data = []
    jiHeJingJia_data = []

    for entry in all_data:
        # 获取交易时间并转换为 datetime 格式
        time_int = entry['t']

        try:
            # 将秒数转换为 HH:MM:SS 格式
            time_str = str(time_int).zfill(6)  # 补充前导零，确保时间格式为六位数
            time_str = f"{time_str[:2]}:{time_str[2:4]}:{time_str[4:]}"  # 格式化为 HH:MM:SS
            time_obj = datetime.strptime(time_str, "%H:%M:%S")

            # 集合竞价的时间段是 9:15:00 到 9:25:00
            if time_obj >= datetime.strptime("09:15:00", "%H:%M:%S") and time_obj <= datetime.strptime("09:25:00", "%H:%M:%S"):
                jiHeJingJia_data.append(entry)

            # 集合竞价的时间段是 9:15:00 到 9:25:00
            if time_obj >= datetime.strptime("09:25:00", "%H:%M:%S") and time_obj <= datetime.strptime("15:00:00", "%H:%M:%S"):
                auction_data.append(entry)
        except ValueError:
            continue

    return jiHeJingJia_data, auction_data

def save_data_to_csv(stock_id, data, filename):
    """将交易数据保存到 CSV 文件"""
    """
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # 写入表头
        writer.writerow(['时间', '成交价', '成交量', '买卖方向'])

        # 写入每一笔交易的数据
        for entry in data:
            print(entry)
            time = entry['t']  # 时间
            price = entry['p'] / 1000 # 成交价
            volume = entry['v']  # 成交量
            buySell = entry['bs']  # 买卖方向 1：买，2：卖 4：集合竞价
            writer.writerow([time, price, volume, buySell])  # 示例调用
    """
    global STOCK_TRACKER
    #STOCK_TRACKER = {}
    """
    STOCK_TRACKER[stock_id] = {
                "stock_id": stock_id,
                "preClosePrice": preClosePrice,
                "time": 0,
                "high": 0.00,
                "low": 0.00,
                "high_updates": 0,
                "low_updates": 0,
            }
    """

    """监控股票价格更新"""
    stock_data = STOCK_TRACKER[stock_id]
    #print(stock_data)
    TOP_OPEN_PERCENT12 = 7
    TOP_OPEN_PERCENT3 = 10

    trade_count = 0

    base_high_count = 0
    base_high_time = "92500"
    base_last_high_count = 0
    base_last_high_time = "92500"

    high_count = 0
    high_time = "92500"
    last_high_count = 0
    last_high_time = "92500"

    sound_count = 0
    sell_count = 0
    last_sell_high_count = 0
    sell_high_count = 0

    last_high = 0.00

    ZTPrice = stock_data["ZTPrice"]
    preClosePrice = stock_data["yesPrice"]
    liutong  = float(stock_data["liutong"])

    #tick_with_minute = getNetAmoutByMin(stock_id, preClosePrice)
    NetAmoutByMin = getNetAmoutByMin(stock_id)
    #print(NetAmoutByMin)
    NetAmoutByTime = 0

    StockName = stock_data["stockName"]
    openPercent = float(stock_data["openPercent"])
    stockMarket = int(stock_data["stockMarket"])
    PanQianUpFlag = stock_data["PanQianUpFlag"]
    PanQianZTSec = int(stock_data["PanQianZTSec"])
    NetBuyAmount = float(stock_data["NetBuyAmount"])

    is_new_stock = (StockName.startswith("N") or StockName.startswith("C"))
    is_high_open = (stockMarket == 3 and openPercent >= TOP_OPEN_PERCENT3) or (stockMarket < 3 and openPercent >= TOP_OPEN_PERCENT12)
    is_low_open = (stockMarket == 3 and openPercent < TOP_OPEN_PERCENT3) or (stockMarket < 3 and openPercent < TOP_OPEN_PERCENT12)

    if PanQianZTSec >= 9:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">股票代码 {stock_id} 集合竞价期间涨停秒数: {PanQianZTSec}</span>'))
    else:
        print(f"股票代码 {stock_id} 集合竞价期间涨停秒数为: {PanQianZTSec}")

    if PanQianUpFlag == "Y":
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">股票代码 {stock_id} 在9:25时高开</span>'))
    else:
        print(f"股票代码 {stock_id}  9:25时低开")

    if openPercent >= 7:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">股票代码 {stock_id} 开盘涨幅为: {openPercent:.2f}%</span>'))
    else:
        print(f"股票代码 {stock_id} 开盘涨幅为: {openPercent:.2f}%")

    if stock_data["is_smooth_trend"] == True:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">股票代码 {stock_id} 集合竞价期间走势平稳</span>'))

    currentPrice = float(stock_data["stockPrice"])
    liutongAmount = int(liutong * currentPrice / 10000)

    if liutongAmount <= 20:
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">股票代码 {stock_id} 当前流通市值为: {liutongAmount}亿</span>'))
    else:
        print(f"股票代码 {stock_id} 当前流通市值为: {liutongAmount}亿")


    stars = ""

    if PanQianZTSec > 1:
        stars = stars + "⭐️"
    if PanQianZTSec > 240:
         stars = stars + "⭐️"

    if PanQianUpFlag == "Y":
        stars = stars + "⭐️"

    if is_high_open:
        stars = stars + "⭐️"

    if liutongAmount <= 20:
        stars = stars + "⭐️"

    if stock_data["is_smooth_trend"] == True:
        stars = stars + "⭐️"

    #STOCK_TRACKER[stock_id]["stars"] = stars
    stock_data.update({
                "stars": stars,})

    print(f"{StockName}（{stock_id}）评级：" +  stars)
    print("--------------------------------------------------")

    if stars:
        stars = stars + "|"

    for trade in data:
        trade_count = trade_count + 1

        time = int(trade['t'])  # 时间
        price = trade['p'] / 1000  # 成交价
        price = float(price)
        liutongAmount = liutong * price / 10000
        #NetAmoutByTime = float(getNetAmoutByTime(tick_with_minute, time))
        #NetAmoutByTime = float(getNetAmoutByTime(NetAmoutByMin, time))

        NetAmoutByTime = float(get_nearest_value(NetAmoutByMin, (time // 100)))  # 输出 200（选择 931 的值）

        percent = (price - preClosePrice) / preClosePrice * 100
        is_high_now= (stockMarket == 3 and percent >= TOP_OPEN_PERCENT3) or (stockMarket < 3 and percent >= TOP_OPEN_PERCENT12)

        #if percent < 2.0:
        if percent < 0.0:
            #STOCK_TRACKER.pop(stock_id)
            #print(f"警告⚠️：股票 {StockName}（{stock_id}）: 在 {time} 涨幅低于2.5% {price}（涨幅：{percent:.2f}%），卖出信号触发，停止追踪")
            print(f"警告⚠️：股票 {StockName}（{stock_id}）: 在 {time} 涨幅低于2.5% {price}（涨幅：{percent:.2f}%）， 机构净买额 {NetAmoutByTime:.0f}万, 卖出信号触发")

            #return
        # 9:25 初始化
        if time == 92500:
            stock_data.update({
                "time": time,
                "high": price,
                "baseline_high": price,
                #"second_high": price,
                "low": price,
                "baseline_low": price,
                #"second_low": price,
                "high_updates": 0,
                "low_updates": 0,
                "buytime": "",
                "buyprice": 0.00,
                "buyFlag": "",
            })
            # 记录最高点
            last_high = price # 设定初始最高价


            #if percent < 3.5:
            if percent < 1.5:
                #STOCK_TRACKER.pop(stock_id)
                #print(f"警告⚠️：股票 {StockName}（{stock_id}）: 在 {time} 涨幅低于3.5% {price}（涨幅：{percent:.2f}%），卖出信号触发，停止追踪")
                #return
                print(f"警告⚠️：股票 {StockName}（{stock_id}）: 在 {time} 涨幅低于1.5% {price}（涨幅：{percent:.2f}%）， 机构净买额 {NetAmoutByTime:.0f}万, 卖出信号触发")


        # 记录 9:40 前的基准最低价和最高价
        #print(openPercent)
        #print( (openPercent >= 10 & time < 93100))
        #print(stockMarket < 3 and openPercent >= TOP_OPEN_PERCENT12)
        """
        # 执行逻辑
        if (
            (time >= 92500)
        and (
                (
                    ((not StockName.startswith("N"))
                      and
                        ((stockMarket == 3 and openPercent >= TOP_OPEN_PERCENT3)
                     or (stockMarket < 3 and openPercent >= TOP_OPEN_PERCENT12))
                    ) and time < 93100
                )
            or  (
                     ((StockName.startswith("N"))
                       or
                        (stockMarket == 3 and openPercent < TOP_OPEN_PERCENT3)
                     or (stockMarket < 3 and openPercent < TOP_OPEN_PERCENT12)
                     ) and time <= 94000
                 )
            )):
        """
        is_greater_than_time_25 = (time >= 92500)
        is_lower_than_time_31 = (time <= 93100)
        is_greater_than_time_31 = (time > 93100)
        is_lower_than_time_40 = (time <= 94000)
        is_greater_than_time_40 = (time > 94000)

        if is_greater_than_time_25:

            if stock_data["baseline_low"] is None or price < stock_data["baseline_low"]:
                stock_data["baseline_low"] = price
                stock_data["baseline_high"] = price  # 重置基准最高价
                stock_data["base_high_updates"] = 0
        """
        if is_greater_than_time_25 and (
            (not is_new_stock and is_high_open and is_lower_than_time_31)
            or
            ((is_new_stock or is_low_open) and is_lower_than_time_31)
        ):
        """
        if is_greater_than_time_25 and (
            (is_high_open and is_lower_than_time_31)
            or
            (is_low_open and is_lower_than_time_31)
        ):
        #if is_greater_than_time_25 and (
        #    (not is_new_stock and is_high_open and is_lower_than_time_31)
        #    or
        #    ((is_new_stock or is_low_open) and is_lower_than_time_31)
        #):
        #if is_greater_than_time_25 and  is_lower_than_time_40:
            #print(f"time={time},{(openPercent >= 10 and time < 93100)}base_last_high_count={base_last_high_count},base_high_count={base_high_count}")
            #print((openPercent >= 10 & time < 93100))
            #if stock_data["baseline_low"] is None or price < stock_data["baseline_low"]:
            #    stock_data["baseline_low"] = price
            #    stock_data["baseline_high"] = None  # 重置基准最高价
            #    stock_data["base_high_updates"] = 0
            if stock_data["baseline_low"] is not None and (stock_data["baseline_high"] is None or price > stock_data["baseline_high"]):
                stock_data["baseline_high"] = price

        # 9:40 后的逻辑
        #elif time > 94000:
        #"""
        #elif (
        #    (not is_new_stock and is_high_open and is_greater_than_time_31)
        #    or
        #    ((is_new_stock or is_low_open) and is_greater_than_time_40)
        #):
        #"""
        """
        if (
            (not is_new_stock and is_high_open and is_greater_than_time_31)
            or
            ((is_new_stock or is_low_open) and is_greater_than_time_31)
        ):
        """
        if (
            (is_high_open and is_greater_than_time_31)
            or
            (is_low_open and is_greater_than_time_31)
        ):
            if price > stock_data["baseline_high"]:
                base_high_count = trade_count
                #print(f"base_last_high_count={base_last_high_count},base_high_count={base_high_count}")
                #if ((base_last_high_count + 5) < base_high_count):
                #    stock_data["base_high_updates"] += 1
                if ((base_last_high_count + 5) < base_high_count) and (seconds_between(str(base_last_high_time), str(time)) > 30):
                    stock_data["base_high_updates"] += 1
                base_high_time = time

                stock_data["baseline_high"] = price
                if stock_data["base_high_updates"] >= 3:
                    print(stars + "提示" + "👍" * stock_data['base_high_updates'] + f"：股票 {StockName}（{stock_id}）: 在 {time} 开盘后最高价更新为 {price}（涨幅：{percent:.2f}%），机构净买额 {NetAmoutByTime:.0f}万,{stock_data['base_high_updates']} 次创新高，买入信号触发")
                else:
                    print(stars + "👍" * stock_data['base_high_updates'] + f"股票 {StockName}（{stock_id}）: 在 {time} 开盘后最高价更新为 {price}（涨幅：{percent:.2f}%），机构净买额 {NetAmoutByTime:.0f}万,{stock_data['base_high_updates']} 次创新高")
                """
                if (price >= stock_data["high"]) and (sound_count == 0) and (PanQianZTSec > 240) and is_high_now and stock_data["is_smooth_trend"] == True  and ((stock_data["base_low_updates"] == 0 and  ((stock_data["base_high_updates"] == 2) or (is_high_open and stock_data["base_high_updates"] == 1))) or (stock_data["base_low_updates"] >= 1 and stock_data["base_high_updates"] == 1)):
                   # 数字到中文的映射
                    chinese_mapping = {
                        '0': '零', '1': '一', '2': '二', '3': '三', '4': '四',
                        '5': '五', '6': '六', '7': '七', '8': '八', '9': '九'
                    }
                    # 将股票代码中的每个数字转换为中文
                    chinese_code = ''.join(chinese_mapping[digit] for digit in str(stock_id))
                    text = f"股票代码 {chinese_code}, 名称 {StockName}, 高开， 当前价格 {price} 元, 涨跌幅 {percent:.2f}, {stock_data['base_high_updates']} 次创新高，赶快设置超过这波最高价的条件买单吧！"

                    print("🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥")
                    #print(text)
                    #print("\033[1;31;43m" + text + "\033[0m")
                    generate_and_play_stock_sounds_fromText(stock_id, text)
                    print("🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥")
                    stock_data["buytime"] = time
                    stock_data["buyprice"] = price

                    sound_count = sound_count + 1
                """
                #if (price >= stock_data["high"]) and (sound_count == 0) and (
                #    ((time <= 93500) and (NetAmoutByTime >= 3000)) or
                #    ((time < 94000) and (NetAmoutByTime >= 5000))
                # ) and (stock_data["base_high_updates"] >= 1):

                #if (price >= stock_data["high"]) and (sound_count == 0) and (
                #    (is_high_open and (time < 94000) and (liutongAmount <= 50) and (PanQianZTSec >= 9) and (NetAmoutByTime >= 1000)) or
                #    ((time <= 93500) and (NetAmoutByTime >= 3000)) or
                #    ((time < 94000) and (NetAmoutByTime >= 5000))
                #) and (stock_data["base_high_updates"] >= 1):
                if (not is_new_stock) and (price >= stock_data["high"]) and (sound_count == 0) and (
                    (is_high_open and (time < 94000) and (liutongAmount <= 50) and (PanQianZTSec >= 9) and (NetAmoutByTime >= 1000)) or
                    ((time < 94000) and (liutongAmount <= 30) and (NetAmoutByTime >= 3000)) or
                    ((time < 94000) and (liutongAmount <= 100) and (NetAmoutByTime >= 5000)) or
                    ((time < 94000) and (NetAmoutByTime >= 8000))
                ) and (stock_data["base_high_updates"] >= 1):

                   # 数字到中文的映射
                    chinese_mapping = {
                        '0': '零', '1': '一', '2': '二', '3': '三', '4': '四',
                        '5': '五', '6': '六', '7': '七', '8': '八', '9': '九'
                    }
                    # 将股票代码中的每个数字转换为中文
                    chinese_code = ''.join(chinese_mapping[digit] for digit in str(stock_id))
                    text = f"股票代码 {chinese_code}, 名称 {StockName}, 高开， 当前价格 {price} 元, 涨跌幅 {percent:.2f}, 机构净买额 {NetAmoutByTime:.0f}万, {stock_data['base_high_updates']} 次创新高，赶快设置超过这波最高价的条件买单吧！"

                    print("🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥")
                    #print(text)
                    #print("\033[1;31;43m" + text + "\033[0m")
                    generate_and_play_stock_sounds_fromText(stock_id, text)
                    print("🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥")
                    stock_data["buytime"] = time
                    stock_data["buyprice"] = price

                    sound_count = sound_count + 1

            if price < stock_data["baseline_low"]:
                stock_data["base_low_updates"] += 1
                stock_data["baseline_low"] = price
                if stock_data["base_low_updates"] >= 1:
                    #print(f"警告⚠️：股票 {StockName}（{stock_id}）: 在 {time} 最低价更新为 {price}（涨幅：{percent:.2f}%），卖出信号触发，停止追踪")
                    print(f"警告⚠️：股票 {StockName}（{stock_id}）: 在 {time} 最低价更新为 {price}（涨幅：{percent:.2f}%），机构净买额 {NetAmoutByTime:.0f}万, 卖出信号触发")
                    #STOCK_TRACKER.pop(stock_id)
                    #return

        # 更新最高价
        #if (stock_data["high"] <= stock_data["baseline_high"]):
            #continue


        #if (time >= 92500) and ((price > stock_data["high"]) or (price == ZTPrice)):
        if (time >= 92500) and (price > stock_data["high"]):
            #stock_data["baseline_low"] = price
            #stock_data["baseline_high"] = price  # 重置基准最高价
            #stock_data["base_high_updates"] = 0
            """
            if is_greater_than_time_31:
                sell_high_count = trade_count
                if ((last_sell_high_count + 5) < sell_high_count): # and (seconds_between(str(last_sell_high_time), str(time)) > 30):
                    last_high = stock_data["high"]
                    #last_sell_high_count = sell_high_count
                print(f"{StockName}（{stock_id}） 在{stock_data['time']} 前一个卖出最高价更新为 {last_high}")
            """

            stock_data["time"] = time
            stock_data["high"] = price



            #if  (
            #    (is_high_open and is_greater_than_time_31)
            #    or
            #    (is_low_open and is_greater_than_time_40)
            #):
            if  (
                ((not is_new_stock) and is_high_open and is_greater_than_time_31)
                or
                ((is_new_stock or is_low_open) and is_greater_than_time_40)
            ):
                high_count = trade_count
                #if (last_high_count + 5) < high_count:
                #print(f"last_high_time:{last_high_time};high_time:{high_time};time:{time};")
                if ((last_high_count + 5) < high_count) and (seconds_between(str(last_high_time), str(time)) > 30):
                    stock_data["high_updates"] += 1

                stock_data["buyFlag"]  =  ("♦️" if stock_data["is_smooth_trend"] else "⛳️") * stock_data['high_updates']
                high_time = time
                print(stars +  stock_data["buyFlag"] + f"{StockName}（{stock_id}） 在{stock_data['time']} 最高价更新为 {price}（涨幅：{percent:.2f}%）机构净买额 {NetAmoutByTime:.0f}万,（{stock_data['high_updates']} 次）")
                #if (sound_count == 0) and  PanQianUpFlag == "Y" and PanQianZTSec > 1 and ((stock_data["high_updates"] == 2) or  (is_high_open and stock_data["high_updates"] == 1)):
                #if (sound_count == 0) and PanQianZTSec > 240 and ((stock_data["high_updates"] == 2) or  (is_high_open and stock_data["high_updates"] == 1) or  (stock_data["low_updates"] >= 1 and stock_data["high_updates"] == 1)):
                #if (sound_count == 0) and  PanQianUpFlag == "Y" and PanQianZTSec > 240 and ((stock_data["high_updates"] == 2) or  (is_high_open and stock_data["high_updates"] == 1) or  (stock_data["low_updates"] >= 1 and stock_data["high_updates"] == 2)):
                #if (sound_count == 0) and PanQianUpFlag == "Y" and PanQianZTSec > 240 and ((stock_data["high_updates"] == 2) or  (is_high_open and stock_data["high_updates"] == 1) or  (stock_data["low_updates"] >= 1 and stock_data["high_updates"] == 1)):
                #if (NetBuyAmount >= 0) and (sound_count == 0) and (PanQianZTSec > 240) and is_high_now and stock_data["is_smooth_trend"] == True  and ((stock_data["low_updates"] == 0 and  ((stock_data["high_updates"] == 2) or (is_high_open and stock_data["high_updates"] == 1))) or (NetBuyAmount >= 5000 and stock_data["low_updates"] >= 1 and stock_data["high_updates"] == 1)):
                #if (sound_count == 0) and (is_new_stock or ((NetBuyAmount >= 0) and (PanQianZTSec > 240) and is_high_now))  and ((stock_data["low_updates"] == 0 and  ((stock_data["high_updates"] == 2) or (is_high_open and stock_data["high_updates"] == 1))) or (NetBuyAmount >= 5000 and stock_data["low_updates"] >= 1 and stock_data["high_updates"] == 1)):

                if should_trigger_alert(sound_count, is_new_stock, NetBuyAmount, PanQianZTSec, is_high_now, is_high_open, stock_data): # 执行你的提醒逻辑
                    # 数字到中文的映射
                    chinese_mapping = {
                        '0': '零', '1': '一', '2': '二', '3': '三', '4': '四',
                        '5': '五', '6': '六', '7': '七', '8': '八', '9': '九'
                    }
                    # 将股票代码中的每个数字转换为中文
                    chinese_code = ''.join(chinese_mapping[digit] for digit in str(stock_id))
                    text = f"股票代码 {chinese_code}, 名称 {StockName}, 高开， 当前价格 {price} 元, 涨跌幅 {percent:.2f}, 机构净买额 {NetAmoutByTime:.0f}万, {stock_data['high_updates']} 次创新高，赶快设置超过这波最高价的条件买单吧！"

                    print("🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥")
                    #print(text)
                    #print("\033[1;31;43m" + text + "\033[0m")
                    generate_and_play_stock_sounds_fromText(stock_id, text)
                    print("🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥")
                    stock_data["buytime"] = time
                    stock_data["buyprice"] = price

                    sound_count = sound_count + 1
                #if (stock_data["high_updates"] >= 3) or  (not is_new_stock and is_high_open and stock_data["low_updates"] == 0  and stock_data["high_updates"] >= 1):
                #if (stock_data["high_updates"] >= 3) or  (is_high_open and stock_data["low_updates"] == 0  and stock_data["high_updates"] >= 1):
                #if (
                #    stock_data["high_updates"] >= 2
                #    or (
                #        (not is_new_stock)
                #        and (
                #            (is_high_open and stock_data["low_updates"] == 0 and stock_data["high_updates"] >= 1)
                #            or (is_low_open and PanQianZTSec >= 9 and stock_data["high_updates"] >= 1)
                #        )
                #    )
                #):
                if (
                    stock_data["high_updates"] >= 2
                    or (
                        (not is_new_stock)
                        and (
                            (is_high_open and stock_data["low_updates"] == 0 and stock_data["high_updates"] >= 1)
                            or (is_low_open and PanQianZTSec >= 9 and stock_data["high_updates"] >= 1)
                            )
                       )
                    or (
                        is_new_stock
                        and  (is_high_open and stock_data["high_updates"] >= 1)
                       )
                ):

                    #stars = ""
                    #if PanQianZTSec > 1:
                        #stars = stars + "⭐️"
                        #if PanQianUpFlag == "Y":
                        #    stars = stars + "⭐️"
                        #if PanQianZTSec > 240:
                        #    stars = stars + "⭐️"
                        #if stock_data["high_updates"] >= 3:
                        #    stars = stars + "⭐️"
                        #if is_high_open:
                        #    stars = stars + "⭐️"
                        #if stock_data["low_updates"] >= 1:
                        #    stars = stars + "⭐️"

                    if (
                        stock_data["high_updates"] >= 3
                        or (
                            (not is_new_stock)
                            and (
                                (is_high_open and stock_data["low_updates"] == 0 and stock_data["high_updates"] >= 1)
                                or (is_low_open and PanQianZTSec >= 9 and stock_data["high_updates"] >= 2)
                                )
                           )
                        or (
                            is_new_stock
                            and  (is_high_open and stock_data["high_updates"] >= 2)
                           )
                    ):

                        print(stars +   ("♦️" if stock_data["is_smooth_trend"] else "⛳️") * stock_data['high_updates'] + f"提示👍🌸🔥🍎🍭🏆✈️： {StockName}（{stock_id}）冲破最高价，买入！")

                    if stock_data["buytime"] == "":
                        stock_data["buytime"] = time
                        stock_data["buyprice"] = price

        #if  is_greater_than_time_31 and (sell_count <= 5 and price < last_high):
        #    print(f"🍏🍏🍏{StockName}（{stock_id}） 在{stock_data['time']} 低于前一个最高价 {last_high}，反弹卖出吧！！！")
        #    sell_count = sell_count + 1

        # 更新最低价
        if (time >= 92500) and (price < stock_data["low"]):
            stock_data["time"] = time
            stock_data["low"] = price

            """
            if (
                (not is_new_stock and is_high_open and is_greater_than_time_31)
                or
                ((is_new_stock or is_low_open) and is_greater_than_time_40)
               ):
            """
            if (
                (is_high_open and is_greater_than_time_31)
                or
                (is_low_open and is_greater_than_time_40)
               ):
                stock_data["low_updates"] += 1
                #print(f"{StockName}（{stock_id}）在{stock_data['time']} 最低价更新为 {price}（涨幅：{percent:.2f}%）（{stock_data['low_updates']} 次）")
                if stock_data["low_updates"] >= 1:
                    #print(f"警告⚠️： {StockName}（{stock_id}）破最低价，不买 or 卖出（涨幅：{percent:.2f}%），移除监控队列")
                    print(f"警告⚠️： {StockName}（{stock_id}）在 {time} 破最低价，不买 or 卖出 {price}（涨幅：{percent:.2f}%），机构净买额 {NetAmoutByTime:.0f}万,反弹立刻卖掉")
                    #STOCK_TRACKER.pop(stock_id)
                    #return
        last_sell_high_count = sell_high_count
        #last_sell_high_time = sell_high_time

        last_high_count = high_count
        last_high_time = high_time
        base_last_high_count = base_high_count
        base_last_high_time = base_high_time


"""
STOCK_TRACKER = {
    "301602":{
                "stockMarket": 3,
                "stockName": "N超研",
                "yesPrice": 6.70,
                "openPercent": 30.00,
                "baseline_low": None,  # 基准最低价
                "baseline_high": None, # 基准最高价                "time": 0,
                "high": 0.00,
                "low": 0.00,
                "base_high_updates": 0,
                "high_updates": 0,
                "base_low_updates": 0,
                "low_updates": 0,
            }
}

stock_id = "301602"


STOCK_TRACKER = {
    "002730":{
                "stockMarket": "2",
                "stockName": "电光科技",
                "yesPrice": 20.69,
                "openPercent": 10.00,
                "baseline_low": None,  # 基准最低价
                "baseline_high": None, # 基准最高价                "time": 0,
                "high": 0.00,
                "low": 0.00,
                "base_high_updates": 0,
                "high_updates": 0,
                "base_low_updates": 0,
                "low_updates": 0,
            }
}

stock_id = "002730"



# 获取所有分时数据
all_data = get_all_fenshi_data_with_auction(stock_id, get_beijing_time())
print(all_data)
# 筛选集合竞价期间的数据
auction_data = filter_auction_data(all_data)
print(auction_data)


# 如果获取到数据，保存到 CSV 文件
if auction_data:
    save_data_to_csv(stock_id, auction_data, 'auction_data.csv')
    #threeUpFromLowest(stock_id, auction_data)
    #print("集合竞价数据已保存到 'auction_data.csv'")
    print(STOCK_TRACKER)

"""

'\nSTOCK_TRACKER = {\n    "301602":{\n                "stockMarket": 3,\n                "stockName": "N超研",\n                "yesPrice": 6.70,\n                "openPercent": 30.00,\n                "baseline_low": None,  # 基准最低价\n                "baseline_high": None, # 基准最高价                "time": 0,\n                "high": 0.00,\n                "low": 0.00,\n                "base_high_updates": 0,\n                "high_updates": 0,\n                "base_low_updates": 0,\n                "low_updates": 0,\n            }\n}\n\nstock_id = "301602"\n\n\nSTOCK_TRACKER = {\n    "002730":{\n                "stockMarket": "2",\n                "stockName": "电光科技",\n                "yesPrice": 20.69,\n                "openPercent": 10.00,\n                "baseline_low": None,  # 基准最低价\n                "baseline_high": None, # 基准最高价                "time": 0,\n                "high": 0.00,\n                "low": 0.00,\n                "base_high_updates": 0,\n                "high_updat

In [219]:
#from google.colab import drive
#drive.mount('/content/drive')

In [220]:
def remove_consecutive_duplicates(price_series):
    if not price_series:
        return []

    result = [price_series[0]]  # 先添加第一个元素
    for price in price_series[1:]:
        if price != result[-1]:  # 只有当当前价格与上一个不同，才添加
            result.append(price)
    return result

# 示例数据
#price_series = [16.44, 16.44, 16.44, 16.42, 16.42, 16.39, 16.38, 16.3, 16.24]
#filtered_prices = remove_consecutive_duplicates(price_series)
#print(filtered_prices)

In [221]:

import numpy as np

def is_smooth_trend(data, stock_id, threshold=1): #, stable_ratio=0.9):
    """
    判断股票价格曲线是否平滑。

    参数：
    - stock_id: 股票 ID，用于存储结果到 STOCK_TRACKER。
    - threshold: 允许的趋势变化次数阈值，默认值为 1。
    - stable_ratio: 价格稳定比例，默认 0.9（如果 90% 以上的数据点相同，则认为平滑）。

    返回：
    - 是否平滑（True/False）
    """
    # 提取价格数据
    price_series = [float(trade['p']) / 1000 for trade in data]
    #print(price_series)

    stock_data = STOCK_TRACKER[stock_id]
    filtered_prices = remove_consecutive_duplicates(price_series)
    #print(filtered_prices)

    """
    # 价格点少于 3 个，直接判定平滑
    if len(filtered_prices) < 3:
        stock_data["is_smooth_trend"] = True
        print("价格曲线是否平滑: True")  # 输出: True
        return #True

    # 计算去重后的价格比例
    unique_ratio = len(set(filtered_prices)) / len(filtered_prices)
    if unique_ratio <= (1 - stable_ratio):
        stock_data["is_smooth_trend"] = True
        print("价格曲线是否平滑: True") # 输出: True
        return #True
    """

    # 计算相邻价格变化
    diffs = np.diff(filtered_prices)
    #print(diffs)

    # 获取趋势方向（上升+1，下降-1，不变0）
    trend = np.sign(diffs)

    # 计算趋势变化次数
    trend_changes = np.sum(np.diff(trend) != 0)

    stock_data["is_smooth_trend"] = (trend_changes <= threshold)

    print(f"价格曲线是否平滑: {trend_changes <= threshold}")  # 输出: True

    # 返回是否平滑
    #return trend_changes <= threshold

In [222]:
def processSTOCK_TRACKER_daily():

    global STOCK_TRACKER

    #print(STOCK_TRACKER)

    """
    STOCK_TRACKER["300766"] = {
                "stockMarket": "2",
                "stockName": "每日互动",
                "yesPrice": 34.70,
                "openPercent": 4.87,
                "baseline_low": None,  # 基准最低价
                "baseline_high": None, # 基准最高价                "time": 0,
                "high": 0.00,
                "low": 0.00,
                "base_high_updates": 0,
                "high_updates": 0,
                "base_low_updates": 0,
                "low_updates": 0,
            }
    """

    try:

        for stock_id, row in list(STOCK_TRACKER.items()):
            print("--------------------------------------------------")
            print(f"Stock ID: {stock_id}")

            ZTCount = get_limit_up_stats_ndays(stock_id)

            STOCK_TRACKER[stock_id]["total_days"] = ZTCount["total_days"]
            STOCK_TRACKER[stock_id]["ZT_days"] = ZTCount["limit_up_days"]
            STOCK_TRACKER[stock_id]["ZT_ratio"] = ZTCount["limit_up_ratio"]

            rank = get_stock_popularity_rank(stock_id)
            STOCK_TRACKER[stock_id]["rank"] = rank

            #preClosePrice = row['yesPrice']


            # 获取所有分时数据
            all_data = get_all_fenshi_data_with_auction(stock_id, get_beijing_time())

            if all_data is None:
                continue
            #print(all_data)
            # 筛选集合竞价期间的数据
            jiHeJingJia_data, auction_data = filter_auction_data(all_data)
            #print(auction_data)
            if jiHeJingJia_data:
                is_smooth_trend(jiHeJingJia_data, stock_id)

            # 如果获取到数据，保存到 CSV 文件
            if auction_data:
                save_data_to_csv(stock_id, auction_data, 'auction_data.csv')
                #threeUpFromLowest(stock_id, auction_data)
                #print("集合竞价数据已保存到 'auction_data.csv'")
                if stock_id in STOCK_TRACKER:
                    print("--------------------------------------------------")
                    print("股票 {}（{}）: 开盘涨幅 {:.2f}%  最高价 {} 最低价 {}".format(
                        STOCK_TRACKER[stock_id]["stockName"], stock_id,
                        STOCK_TRACKER[stock_id]["openPercent"],
                        STOCK_TRACKER[stock_id]["high"], STOCK_TRACKER[stock_id]["low"]
                    ))
                    print("--------------------------------------------------")
            #print(STOCK_TRACKER)

    except Exception as e:
        import traceback
        print(f"Error: {e}")
        print(traceback.format_exc())

In [223]:





#!pip install xlwings
#!pip install pandas

import os
#import xlwings as xw
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import urllib.request

#from re import compile
from time import ctime
import datetime
nowTime=datetime.datetime.now().strftime('%H:%M:%S')
#print(nowTime)

rootDir = r'D:\Stock\Million'


#install_packages(["pandas", "xlwings"])
"""
@xw.func
@xw.arg('ID', np.array, ndim=1)
@xw.arg('Price', np.array, ndim=1)
@xw.arg('CurrentPerc', np.array, ndim=1)
@xw.arg('NetBuyAmount', np.array, ndim=1)
@xw.arg('NetBuyPerc', np.array, ndim=1)
@xw.arg('NetPerCirculationMarketValue', np.array, ndim=1)
@xw.arg('StartPerc', np.array, ndim=1)
@xw.arg('ExchangeRate', np.array, ndim=1)
@xw.arg('LiangBi', np.array, ndim=1)
@xw.arg('Liutong', np.array, ndim=1)
@xw.arg('TopPerc', np.array, ndim=1)
@xw.arg('LowPerc', np.array, ndim=1)
@xw.arg('MarketHours', np.array, ndim=1)
@xw.ret(index=False, header=False)
"""
def machine_learn_pred(ID, Price, CurrentPerc, NetBuyAmount, NetBuyPerc, NetPerCirculationMarketValue, StartPerc, ExchangeRate, LiangBi, Liutong, TopPerc, LowPerc, MarketHours):
    import pandas as pd
    import ztop_ai as zai
    #1
    fsr0=rootDir + r'\EarnBillion\dat\ccpp_'
    print('#1',fsr0)
    #mlst1=['log','knn','forest','dtree','mlp']
    mlst1=['log','knn','forest','dtree','gbdt','mlp']

    testDF = pd.DataFrame({"ID": ID, "Price": Price, "CurrentPerc": CurrentPerc, 'NetBuyAmount': NetBuyAmount, 'NetBuyPerc': NetBuyPerc, 'NetPerCirculationMarketValue': NetPerCirculationMarketValue, 'StartPerc': StartPerc, 'ExchangeRate': ExchangeRate, 'LiangBi': LiangBi, 'Liutong': Liutong, 'TopPerc': TopPerc, 'LowPerc': LowPerc, 'MarketHours': MarketHours},
                          columns=["ID", "Price", "CurrentPerc", "NetBuyAmount", "NetBuyPerc", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "TopPerc", "LowPerc", "MarketHours"])
#                          index=[0])
#                      columns=["ID", "Price", "CurrentPerc", "NetBuyAmount", "NetBuyPerc", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "TopPerc", "LowPerc", "MarketDate", "MarketHours"])
#                      index=["ID", "Price", "CurrentPerc", "NetBuyAmount", "NetBuyPerc", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "TopPerc", "LowPerc", "MarketDate", "MarketHours"])

    predictors = ["ID", "Price", "CurrentPerc", "NetBuyAmount", "NetBuyPerc", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "TopPerc", "LowPerc", "MarketHours"]

    testDF[predictors].to_csv(rootDir + r"\EarnBillion\dat\ccpp_xtest.csv", index=False)
#    testDF["Label"].to_csv(rootDir + r"\EarnBillion\dat\ccpp_ytest.csv", index=False, header=True)

    x_train, x_test, y_train, y_test=zai.ai_dat_rd(fsr0)
#    print("x_test", x_test)

#    print("x_test", x_test)
#    y_test = pd.DataFrame([Label],
#                      columns=['numbers'],
#                      index=["Label"])

    #2
    zai.xmodel={}
    print('\n#2,ai_f_mxRdlst')
    zai.ai_f_mxRdlst(fsr0,mlst1)

    #3
    print('\n#3,mlst1',mlst1)
    dacc,df9 = zai.mx_mul(mlst1, x_test,  y_test,yk0=5,fgInt=False,fgDebug=True)

    #-----------------------
    print('\nok!')

    #print(df9['y_sum'])
    return df9['y_sum']

"""
@xw.func
@xw.arg('ID', np.array, ndim=1)
@xw.arg('CurrentPerc', np.array, ndim=1)
@xw.arg('NetBuyAmount', np.array, ndim=1)
@xw.arg('CirculationMarketValue', np.array, ndim=1)
@xw.arg('NetPerCirculationMarketValue', np.array, ndim=1)
@xw.arg('StartPerc', np.array, ndim=1)
@xw.arg('ExchangeRate', np.array, ndim=1)
@xw.arg('LiangBi', np.array, ndim=1)
@xw.arg('Liutong', np.array, ndim=1)
@xw.arg('Good', np.array, ndim=1)
@xw.arg('ZT', np.array, ndim=1)
@xw.arg('ZL', np.array, ndim=1)
@xw.arg('Valued', np.array, ndim=1)
@xw.arg('ML', np.array, ndim=1)
@xw.arg('ZT1', np.array, ndim=1)
@xw.arg('ZT2', np.array, ndim=1)
@xw.arg('ZT3', np.array, ndim=1)
@xw.arg('ZT4', np.array, ndim=1)
@xw.arg('ZT5', np.array, ndim=1)
@xw.arg('ZL1', np.array, ndim=1)
@xw.arg('ZL2', np.array, ndim=1)
@xw.arg('ZL3', np.array, ndim=1)
@xw.arg('ZL4', np.array, ndim=1)
@xw.arg('ZL5', np.array, ndim=1)
@xw.arg('PanQianBelowYes', np.array, ndim=1)
@xw.arg('PanQianZT', np.array, ndim=1)
@xw.arg('PanQianZTSec', np.array, ndim=1)
@xw.arg('PanQianLowPerc', np.array, ndim=1)
@xw.ret(index=False, header=False)
"""
def morning_pred(ID, CurrentPerc, NetBuyAmount, CirculationMarketValue, NetPerCirculationMarketValue, StartPerc, ExchangeRate, LiangBi, Liutong, Good,
    ZT, ZL, Valued, ML, ZT1, ZT2, ZT3, ZT4, ZT5, ZL1, ZL2, ZL3, ZL4, ZL5, PanQianBelowYes, PanQianZT, PanQianZTSec, PanQianLowPerc):
    import pandas as pd
    import ztop_ai as zai
    #1
    fsr0=rootDir + r'\EarnBillion\dat\Accpp_'
    print('#1',fsr0)
    #mlst1=['log','knn','forest','dtree','mlp']
    mlst1=['knn','forest','dtree','gbdt']


    dataDF = pd.DataFrame({"ID": ID, "CurrentPerc": CurrentPerc, 'NetBuyAmount': NetBuyAmount, 'CirculationMarketValue': CirculationMarketValue, 'NetPerCirculationMarketValue': NetPerCirculationMarketValue,
                        'StartPerc': StartPerc, 'ExchangeRate': ExchangeRate, 'LiangBi': LiangBi, 'Liutong': Liutong, 'Good': Good,
                        'ZT': ZT, 'ZL': ZL, 'Valued': Valued, 'ML': ML, 'ZT1': ZT1, 'ZT2': ZT2, 'ZT3': ZT3, 'ZT4': ZT4, 'ZT5': ZT5, 'ZL1': ZL1, 'ZL2': ZL2, 'ZL3': ZL3, 'ZL4': ZL4, 'ZL5': ZL5,
                        'PanQianBelowYes': PanQianBelowYes, 'PanQianZT': PanQianZT, 'PanQianZTSec': PanQianZTSec, 'PanQianLowPerc': PanQianLowPerc },
                        columns=["ID", "CurrentPerc", "NetBuyAmount", "CirculationMarketValue", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "Good",
                        "ZT", "ZL", "Valued", "ML", "ZT1", "ZT2", "ZT3", "ZT4", "ZT5", "ZL1", "ZL2", "ZL3", "ZL4", "ZL5", "PanQianBelowYes", "PanQianZT", "PanQianZTSec", "PanQianLowPerc"])
#                          index=[0])
#                      columns=["ID", "Price", "CurrentPerc", "NetBuyAmount", "NetBuyPerc", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "TopPerc", "LowPerc", "MarketDate", "MarketHours"])
#                      index=["ID", "Price", "CurrentPerc", "NetBuyAmount", "NetBuyPerc", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "TopPerc", "LowPerc", "MarketDate", "MarketHours"])
    print(dataDF)
    dataDF['MarketID'] = ([('{:0>6}'.format(id))[0] for id in dataDF['ID'].tolist()])
    dataDF['MarketID'] = dataDF['MarketID'].astype(int)

    dataDF = dataDF.fillna(0)

    dataDF['Good'] = (dataDF['Good'] == 'Y').astype('int')
    dataDF['ZT1'] = (dataDF['ZT1'] == 'Y').astype('int')
    dataDF['ZT2'] = (dataDF['ZT2'] == 'Y').astype('int')
    dataDF['ZT3'] = (dataDF['ZT3'] == 'Y').astype('int')
    dataDF['ZT4'] = (dataDF['ZT4'] == 'Y').astype('int')
    dataDF['ZT5'] = (dataDF['ZT5'] == 'Y').astype('int')
    dataDF['ZL1'] = (dataDF['ZL1'] == 'Y').astype('int')
    dataDF['ZL2'] = (dataDF['ZL2'] == 'Y').astype('int')
    dataDF['ZL3'] = (dataDF['ZL3'] == 'Y').astype('int')
    dataDF['ZL4'] = (dataDF['ZL4'] == 'Y').astype('int')
    dataDF['ZL5'] = (dataDF['ZL5'] == 'Y').astype('int')
    dataDF['PanQianBelowYes'] = (dataDF['PanQianBelowYes'] == 'Y').astype('int')
    dataDF['PanQianZT'] = (dataDF['PanQianZT'] == 'Y').astype('int')


    predictors = ['MarketID', "ID", "CurrentPerc", "NetBuyAmount", "CirculationMarketValue", "NetPerCirculationMarketValue", "StartPerc", "ExchangeRate", "LiangBi", "Liutong", "Good",
                "ZT", "ZL", "Valued", "ML", "ZT1", "ZT2", "ZT3", "ZT4", "ZT5", "ZL1", "ZL2", "ZL3", "ZL4", "ZL5", "PanQianBelowYes", "PanQianZT", "PanQianZTSec", "PanQianLowPerc"]


    dataDF[predictors].to_csv(rootDir + r"\EarnBillion\dat\Accpp_xtest.csv", index=False)
#    dataDF["Label"].to_csv(rootDir + r"\EarnBillion\dat\ccpp_ytest.csv", index=False, header=True)

    x_train, x_test, y_train, y_test=zai.ai_dat_rd(fsr0)
#    print("x_test", x_test)

#    print("x_test", x_test)
#    y_test = pd.DataFrame([Label],
#                      columns=['numbers'],
#                      index=["Label"])

    #2
    zai.xmodel={}
    print('\n#2,ai_f_mxRdlst')
    zai.ai_f_mxRdlst(fsr0,mlst1)

    #3
    print('\n#3,mlst1',mlst1)
    dacc,df9 = zai.mx_mul(mlst1, x_test,  y_test,yk0=5,fgInt=False,fgDebug=True)

    #-----------------------
    print('\nok!')

    #print(df9['y_sum'])
    return df9['y_sum']


"""
@xw.func
@xw.arg('ID', np.array, ndim=1)
@xw.arg('liutong', np.array, ndim=1)
@xw.ret(index=False, header=False)
"""
def getFirstAmountByurls(ID, liutong):
    import pandas as pd
    getMarketID = lambda stockID: "1" if ((('{:0>6}'.format(stockID))[0]) == "6") else "2"
    getMarketID2 = lambda stockID: "1" if ((('{:0>6}'.format(stockID))[0]) == "6") else "0"
    #start_urls = ["http://mdfm.eastmoney.com/EM_UBG_MinuteApi/Js/Get?dtype=all&token=44c9d251add88e27b65ed86506f6e5da&rows=144000&cb=jQuery17206482794749707517_1524799501549&page=1&id=" + '{:0>6}'.format(int(id)) + getMarketID(int(id)) + "&gtvolume=&sort=&_=1524799741547" for id in ID]
    start_urls = ["http://push2ex.eastmoney.com/getStockFenShi?pagesize=144000&ut=7eea3edcaed734bea9cbfc24409ed989&dpt=wzfscj&cb=jQuery1124019826114890089808_1611886439989&pageindex=0&id=" + '{:0>6}'.format(int(id)) + getMarketID(int(id)) + "&sort=1&ft=1&code=" + '{:0>6}'.format(int(id)) + "&market=" + getMarketID2(int(id)) + "&_=1611886439991" for id in ID]
    #print(start_urls)
    liutongPer = [liutongT for liutongT in liutong]
    amountDetails = []
    print('At', ctime(), 'on EastMoney...')
    with ThreadPoolExecutor(4) as executor:
         amountDetails = [detail for detail in (executor.map(getFirstAmount, start_urls, liutongPer))]
    print('all DONE at:', ctime())

#    amountDetails = [getFirstAmount(url) for url in start_urls]
    firstAmount = pd.DataFrame({'NetBuyAmount': pd.Series(amountDetails)})
##    print(firstAmount)
##
    return firstAmount['NetBuyAmount']





def getFirstAmount(url, liutongPer ):
    totalAmount = 0
    totalAmount2 = 0
    totalAmount3 = 0
    PanQianBelowYes = ''
    PanQianZT = ''
    PanQianZTSec = 1

    PanQianLowPrice = 0
    PanQianLowPerc = 0.00
    PanQianNowPerc = 0.00

    PanQianStartPerc = 0.00

    panqianLastAmount = 0
    panqianLastBuyFlag = ""

    bigOrderNumber = 0
    bigOrderNumberPerc = 0.00

    PanQianUpFlag = ""

    minPrice = 0.000
    minTime = "000000"
    minVolume = 0

    minChanceRadio = 0.00

    PanQianVolume = 0.00

    ZTPrice = 0.00

    changeRadioTime70 = "999999"
    changeRadioTime60 = "999999"
    changeRadioTime40 = "999999"
    changeRadioTime5 = "999999"

    firstAmountDetail = [ minChanceRadio,  PanQianVolume, changeRadioTime5,  changeRadioTime40,  changeRadioTime60,   changeRadioTime70,  ZTPrice,   totalAmount, PanQianBelowYes, PanQianZT, PanQianZTSec, PanQianLowPerc, PanQianNowPerc, totalAmount2, totalAmount3, bigOrderNumber, bigOrderNumberPerc, PanQianUpFlag]

    #import urllib.request
    #print(url)
    #urlContext = urllib.request.urlopen(url).read()
    urlContext = downloadFromURL(url)
    # print(urlContext, url)
    if urlContext is None:
        #return totalAmount
        #print("Download Error", url)
        return firstAmountDetail

    strContext = urlContext.decode()
    details = strContext.split('[{')
    #print(details)

    if len(details) <= 1:
        #return totalAmount
        #print("Download Error", url)
        return firstAmountDetail
    #print(((((details[0]).split('"pc":"'))[1]).split('","data":'))[0], url)
    #yesPrice = float(((((details[0]).split('"pc":"'))[1]).split('","data":'))[0])
    #yesPrice = float(((((details[0]).split('"pc":"'))[1]).split('","data":'))[0])
#    totalAmount = 0
    #print('Continue')
    stockID = (((strContext.split('"data":{"c":"'))[1]).split('","m":'))[0]
    #print(stockID)
    marketID = "1" if ((('{:0>6}'.format(stockID))[0]) == "6") else ("3" if ((('{:0>6}'.format(stockID))[0]) == "3") else "2")
    #print(marketID)

    buyFlag = "0"
    panqianLastPrice = 0.00

    panqianLastBuyFlag = ""
    panqianLastPrice = 0.00

    panqianLastBuyFlag25 = ""
    panqianLastPrice25 = 0.00

    sumVolume = 0



    #TimeLimit = "09:25:00"

    if len(details) > 1:
        yesPrice = float(((((details[0]).split('"cp":'))[1]).split(',"tc":'))[0]) / 1000
        #print(yesPrice)
        ZTPrice = yesPrice * (1.10 if (marketID == '1' or marketID == '2') else 1.20)
        #ZTPrice = float('%.2f' % ZTPrice)

        ZTPrice = round(ZTPrice, 2)
        totalAmountPerMin = (((details[1]).split('}]'))[0]).split('},{')
        #print(totalAmountPerMin)
        #print(len(totalAmountPerMin))

        panqianLastPrice = 0
        #panqianLastZTTime = "09:25:00"
        panqianLastZTTime = "092500"

        ZTCount = 0
        UPCount = 0
        #print(url)
        startOrderNum = 0
        if len(totalAmountPerMin) > 0:
            for i in range(len(totalAmountPerMin)):
                #print("i=", i)
                #print(totalAmountPerMin[i])
                amountPerMin = totalAmountPerMin[i].split(',')
                latestAmount = amountPerMin
                # if amountPerMin[0] < "09:16:00":
                #     print(url)
                #     print(amountPerMin)
                minPrice = float((amountPerMin[1].split(':'))[1]) / 1000
                #print("minPrice:" + str(minPrice))
                minTime ='{:0>6}'.format('{:0>6}'.format((amountPerMin[0].split(':'))[1]))
                minVolume = float((amountPerMin[2].split(':'))[1])
                # print(minPrice)
                # print(minTime)
                # print(minVolume)
                if minTime >= "092500":
                    sumVolume = sumVolume + minVolume
                    #print(sumVolume)
                    if sumVolume / liutongPer >= 70:
                       #print( sumVolume / liutongPer)
                       if changeRadioTime70 == "999999":
                            #print(changeRadioTime70)
                            changeRadioTime70 = minTime
                    elif sumVolume / liutongPer >= 60:
                        #print( sumVolume / liutongPer)

                        if changeRadioTime60 == "999999":
                            changeRadioTime60 = minTime
                    elif sumVolume / liutongPer >= 40:
                        #print( sumVolume / liutongPer)

                        if changeRadioTime40 == "999999":
                            changeRadioTime40 = minTime
                    elif sumVolume / liutongPer >= 5:
                        if changeRadioTime5 == "999999":
                            changeRadioTime5 = minTime

                if minTime < "093000":
                    PanQianVolume = minVolume
                    PanQianNowPerc = float('%.2f' % ((minPrice/yesPrice - 1) * 100))
                #if PanQianZT == 'Y' and ZTCount == 0 and (minPrice/yesPrice < 1.09 or (minPrice/yesPrice >= 1.09 and minTime < "093000")) and panqianLastPrice/yesPrice >= 1.09:
                if PanQianZT == 'Y' and ZTCount == 0 and (minPrice < ZTPrice or (minPrice == ZTPrice and minTime >= "092500")):
                    #print(minTime)

                    PanQianZTSec = caculate_time(panqianLastZTTime, minTime)
                    ZTCount = ZTCount + 1

                #if minTime < "092500" and minTime <= TimeLimit:
                if minTime < "092500":
                    #PanQianNowPerc = float('%.2f' % ((minPrice/yesPrice - 1) * 100))
                    #print(amountPerMin, panqianLastPrice, buyFlag)
                    #if minPrice/yesPrice < 1.018:
                    if minPrice/yesPrice < 1.03:
                        PanQianBelowYes = 'Y'
                    #if ZTPrice == minPrice:
                    #    PanQianZT = 'Y'
                    if i == 0 and minTime < "092000":
                        if ((marketID == '1' or marketID == '2') and minPrice/yesPrice >= 1.09) or (marketID == '3' and minPrice/yesPrice >= 1.19):
                        #if minPrice/yesPrice >= 1.08:
                            PanQianZT = 'Y'
                            panqianLastZTTime = minTime
                            #print(panqianLastZTTime)

                    if minPrice > panqianLastPrice and panqianLastPrice != 0:
                        # buyFlag = "1"
                        pass
                    elif minPrice < panqianLastPrice and panqianLastPrice != 0:
                        # buyFlag = "-1"
                        if PanQianLowPrice > minPrice:
                            PanQianLowPrice = minPrice
                            PanQianLowPerc = float('%.2f' % ((PanQianLowPrice/yesPrice - 1) * 100))
                    elif panqianLastPrice == 0:
                        # buyFlag = "0"
                        PanQianLowPrice = minPrice
                        PanQianLowPerc = float('%.2f' % ((PanQianLowPrice/yesPrice - 1) * 100))

                    if (amountPerMin[3].split(':'))[1] == '2':
                        upFlag = 'Y'
                        totalAmount = minPrice * minVolume * 100 / 10000
                        #print(amountPerMin, PanQianLowPerc, upFlag, totalAmount, PanQianNowPerc)
                    elif (amountPerMin[3].split(':'))[1] == '1':
                        upFlag = 'N'
                        totalAmount = minPrice * minVolume * 100 / 10000 * (-1)
                    else:
                        upFlag = ''
                        totalAmount = 0

                    if minTime >= "091700" and PanQianLowPerc >= 2.5:
                        UPCount = UPCount + 1
                        if minPrice >= panqianLastPrice and (UPCount == 1 or PanQianUpFlag == "Y"):
                            PanQianUpFlag = "Y"
                        else:
                            PanQianUpFlag = ""
                    else:
                        PanQianUpFlag = ""


                    panqianLastPrice = minPrice
                    panqianLastAmount = totalAmount
                    panqianLastBuyFlag = upFlag
                    totalAmount2 = totalAmount
                    totalAmount3 = max(totalAmount, totalAmount2, panqianLastAmount)

                    #print(amountPerMin, panqianLastPrice, panqianLastAmount, panqianLastBuyFlag, totalAmount2)

                #elif minTime == "092500" and minTime <= TimeLimit:
                elif minTime == "092500":
                    #print(amountPerMin, panqianLastPrice, buyFlag)
                    # buyFlag = (amountPerMin[3].split(':'))[1]
                    # if minPrice > panqianLastPrice and panqianLastPrice != 0:
                    #     buyFlag = "1"
                    # elif minPrice < panqianLastPrice and panqianLastPrice != 0:
                    #     buyFlag = "-1"
                    # elif panqianLastPrice == 0:
                    #     buyFlag = "0"

                    #panqianLastPrice = minPrice
                    # print(amountPerMin, panqianLastPrice, buyFlag)
                    panqianLastPrice25 = minPrice

                    if panqianLastPrice > panqianLastPrice25: #and PanQianUpFlag == "Y":
                        PanQianUpFlag = ""
                    else:
                        PanQianUpFlag = "Y"

                    if panqianLastPrice25/yesPrice < 1.03:
                        PanQianBelowYes = 'Y'

                    if PanQianLowPrice > panqianLastPrice25:
                        PanQianLowPrice = panqianLastPrice25
                        PanQianLowPerc = float('%.2f' % ((PanQianLowPrice/yesPrice - 1) * 100))

                    if (amountPerMin[3].split(':'))[1] == '2':
                        panqianLastBuyFlag25 = 'Y'
                        #print(amountPerMin, PanQianLowPerc, upFlag, totalAmount, PanQianNowPerc)
                    elif (amountPerMin[3].split(':'))[1] == '1':
                        panqianLastBuyFlag25 = 'N'
                    else:
                        panqianLastBuyFlag25 = ''

                    #print(amountPerMin, panqianLastPrice25, panqianLastBuyFlag25)
                #elif minTime > "092500" and minTime < "093000" and minTime <= TimeLimit:
                elif minTime > "092500" and minTime < "093000":
                    if ((marketID == '1' or marketID == '2') and PanQianLowPerc >= 9.9) or (marketID == '3' and PanQianLowPerc >= 19.9):
                        PanQianZT = 'Y'

                    # print(amountPerMin, panqianLastPrice, buyFlag)
                    #print(panqianLastPrice, latestAmount[1], (amountPerMin[3].split(':'))[1], url)
                    buyFlag = "0"
                    totalAmount = 0
                    totalAmount2 = 0
                    totalAmount3 = 0

                    if panqianLastPrice > minPrice: # and PanQianUpFlag == "Y":
                        PanQianUpFlag = ""
                    else:
                        PanQianUpFlag = "Y"

                    PanQianStartPerc = float('%.2f' % ((minPrice/yesPrice - 1) * 100))

                    if minPrice > panqianLastPrice and panqianLastPrice != 0:
                        buyFlag = "1"
                        totalAmount = minPrice * minVolume * 100 / 10000
                    elif minPrice == panqianLastPrice and panqianLastBuyFlag == 'Y' and panqianLastPrice != 0:
                        buyFlag = "1"
                        totalAmount = minPrice * minVolume * 100 / 10000
                    elif minPrice < panqianLastPrice and panqianLastPrice != 0:
                        buyFlag = "-1"
                        totalAmount = minPrice * minVolume * 100 / 10000 * (-1)
                    elif panqianLastPrice == 0:
                        buyFlag = "0"
                        totalAmount = 0
                    #print(amountPerMin, buyFlag)

                    if (amountPerMin[3].split(':'))[1] == "2":
                        totalAmount2 = minPrice * minVolume * 100 / 10000
                        # if buyFlag == "-1":
                        #     buyFlag = "-1"
                        #     totalAmount = minPrice * minVolume * 100 / 10000 * (-1)
                        # else:
                        #     buyFlag = "1"
                        #     totalAmount = minPrice * minVolume * 100 / 10000
                    elif (amountPerMin[3].split(':'))[1] == "1":
                        totalAmount2 = minPrice * minVolume * 100 / 10000 * (-1)
                        # if minPrice > panqianLastPrice and panqianLastPrice != 0:
                        #     buyFlag = "1"
                        #     totalAmount2 = minPrice * minVolume * 100 / 10000
                        # else:
                        #     buyFlag = "-1"
                        #     totalAmount2 = minPrice * minVolume * 100 / 10000 * (-1)
                    elif (amountPerMin[3].split(':'))[1] == "4":
                        totalAmount2 = 0
                        # if panqianLastBuyFlag == "N":
                        #     totalAmount2 = minPrice * minVolume * 100 / 10000 * (-1)
                        # elif panqianLastBuyFlag == "Y":
                        #     totalAmount2 = minPrice * minVolume * 100 / 10000
                        # else:
                        #     totalAmount2 = 0

                        # if buyFlag == "-1":
                        #     buyFlag = "-1"
                        #     totalAmount = minPrice * minVolume * 100 / 10000 * (-1)
                        # elif buyFlag == "1":
                        #     buyFlag = "1"
                        #     totalAmount = minPrice * minVolume * 100 / 10000
                        # else:
                        #     buyFlag = "0"
                        #     totalAmount = 0
                    if PanQianStartPerc > PanQianLowPerc:
                        totalAmount3 = max(totalAmount, totalAmount2, panqianLastAmount)
                        # if max(abs(totalAmount), abs(totalAmount2), abs(panqianLastAmount)) > totalAmount3:
                        #     totalAmount3 = min(totalAmount, totalAmount2, panqianLastAmount)
                    else:
                        totalAmount3 = min(totalAmount, totalAmount2, panqianLastAmount)
                    #print("totalAmount:" , totalAmount, "totalAmount2:" , totalAmount2, "panqianLastAmount:" , panqianLastAmount, "totalAmount3:" , totalAmount3)

                    if minVolume * 100 >= 100000 or minPrice * minVolume * 100 / 10000 >= 50:
                        bigOrderNumber = 1
                        bigOrderNumberPerc = 100
                    startOrderNum = i
                #elif minTime >= "093000" and minTime <= TimeLimit:
                elif minTime >= "093000":
                    if (amountPerMin[3].split(':'))[1] == "2":
                        totalAmount = totalAmount + minPrice * minVolume * 100 / 10000
                    elif (amountPerMin[3].split(':'))[1] == "1":
                        totalAmount = totalAmount + minPrice * minVolume * 100 / 10000 * (-1)
                    elif (amountPerMin[3].split(':'))[1] == "4":
                        totalAmount = totalAmount + 0

                    if minVolume * 100 >= 100000 or minPrice * minVolume * 100 / 10000 >= 50:
                        bigOrderNumber = bigOrderNumber + 1
                    bigOrderNumberPerc = bigOrderNumber / (i - startOrderNum + 1) * 100
                    #print(startOrderNum, i, bigOrderNumber, bigOrderNumberPerc)
                    #print(url, minTime, bigOrderNumber, bigOrderNumberPerc)
                # if minTime >= "094000":
                #     break
    # if totalAmount > 3000:
    #     print(url, totalAmount)

    # if nowTime < "09:30:00":
    #     #print(nowTime)
    #     totalAmount = max(totalAmount, panqianLastAmount)
    #totalAmount = max(totalAmount, panqianLastAmount)
    #print("PanQianUpFlag=", PanQianUpFlag)
    firstAmountDetail = [ minChanceRadio,  PanQianVolume, changeRadioTime5,  changeRadioTime40,  changeRadioTime60,   changeRadioTime70,  ZTPrice,  totalAmount, PanQianBelowYes, PanQianZT, PanQianZTSec, PanQianLowPerc, PanQianNowPerc, totalAmount2, totalAmount3, bigOrderNumber, bigOrderNumberPerc, PanQianUpFlag]
    #print(firstAmountDetail, totalAmount, PanQianBelowYes, PanQianZT, PanQianZTSec, PanQianLowPerc, PanQianNowPerc, totalAmount2, totalAmount3, bigOrderNumber, bigOrderNumberPerc, PanQianUpFlag)
    #return totalAmount, PanQianBelowYes
    return firstAmountDetail

def caculate_time(time_str_Start, time_str_End):
    #print(time_str_Start, time_str_End)
    time_Start = split_time(time_str_Start)
    time_End = split_time(time_str_End)
    seconds = time_End - time_Start
    #print(seconds)
    return seconds
def split_time(time_str):
    #line = time_str.split(":")
    seconds = int(time_str[0:2])*3600 + int(time_str[2:4])*60 + int(time_str[4:6])
    #print(seconds)
    return seconds
import urllib.request
import urllib.error

def downloadFromURL(url, user_agent='wswp', num_retries=3, timeout=10):
    html = None
    headers = {'User-agent': user_agent}
    request = urllib.request.Request(url, headers=headers)
    try:
        with urllib.request.urlopen(request, timeout=timeout) as response:
            html = response.read()
    except urllib.error.URLError as e:
        print('Download error:', e.reason, url)
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                return downloadFromURL(url, user_agent, num_retries-1, timeout)
    except Exception as e:
        print('Download failed:', e)
    return html

def downloadFromURL0(url, user_agent='wswp', num_retries=3):
    html = None
    #print(':', url)
    # print('Downloading At', ctime())
    hearders = {'User-agent': user_agent}
    request = urllib.request.Request(url, headers=hearders)
    # try:
    #     request = urllib.request.Request(url, headers=hearders)
    #     #print("request Result", request)
    # except:
    #     print('request except')
    #     html = None
    # # else:
    # #     print("request else")
    # # finally:
    # #     print("request finally")

    #print("Test Result", request)
    try:
        html = urllib.request.urlopen(request).read()
        #print("Download Try Result", html)
    except urllib.request.URLError as e:
        print('Download error:', e.reason, url)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                #recursively retry 5xx HTTP errors
                return downloadFromURL0(url, num_retries-1)
    except:
        print('Download except')
        html = None
    # else:
    #     print("Download else")
    # finally:
    #     print("Download finally")


    # print("html len: ", len(html.decode().split('["')))
    # print("html: ", html.decode().split('["'))
    # if len(html.decode().split('["')) <= 1:
    #     html = None
    # print('Downloading End', ctime())
    return html

"""
@xw.func
@xw.arg('ID', np.array, ndim=1)
@xw.ret(index=False, header=False)
"""
def getLatestAmountByurls(ID):
    # import re
    # import urllib.request

    # # The proxy address and port:
    # proxy_info = { 'host' : 'web-proxy.oa.com','port' : 8080 }

    # # We create a handler for the proxy
    # proxy_support = urllib.request.ProxyHandler({"http" : "http://%(host)s:%(port)d" % proxy_info})

    # # We create an opener which uses this handler:
    # opener = urllib.request.build_opener(proxy_support)

    # # Then we install this opener as the default opener for urllib2:
    # urllib.request.install_opener(opener)


    import pandas as pd
    getMarketID = lambda stockID: "1" if ((('{:0>6}'.format(stockID))[0]) == "6") else "0"
    #start_urls = ["http://ff.eastmoney.com/EM_CapitalFlowInterface/api/js?id=" + '{:0>6}'.format(int(id)) + getMarketID(int(id)) + "&type=ff&check=MLBMS&cb=var%20aff_data=&js={(x)}&rtntype=3&acces_token=1942f5da9b46b069953c873404aad4b5&_=1504234842272" for id in ID]
    #start_urls = ["http://ff.eastmoney.com/EM_CapitalFlowInterface/api/js?id=" + '{:0>6}'.format(int(id)) + getMarketID(int(id)) + "&type=ff&check=MLBMS&cb=var%20aff_data=&js={(x)}&rtntype=3&acces_token=1942f5da9b46b069953c873404aad4b5&_=1504234842272" for id in ID]
    start_urls = ["http://push2.eastmoney.com/api/qt/stock/fflow/kline/get?cb=jQuery112305517775202340915_1612147619850&lmt=0&klt=1&fields1=f1%2Cf2%2Cf3%2Cf7&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61%2Cf62%2Cf63%2Cf64%2Cf65&ut=b2884a393a59ad64002292a3e90d46a5&secid="+ getMarketID(int(id)) + "." + '{:0>6}'.format(int(id)) + "&_=1612147619878" for id in ID]
    amountDetails = []
    print('At', ctime(), 'on EastMoney...')
    with ThreadPoolExecutor(4) as executor:
         amountDetails = [detail for detail in (executor.map(getLatestAmount, start_urls))]
    print('all DONE at:', ctime())

    return amountDetails

# #    amountDetails = [getLatestAmount(url) for url in start_urls]
#     LatestAmount = pd.DataFrame({'NetBuyAmount': pd.Series(amountDetails)})
# ##    print(LatestAmount)
# ##
#     return LatestAmount['NetBuyAmount']


def getLatestAmount(url):
    #import urllib.request
    #print(url)
    #urlContext = urllib.request.urlopen(url).read()
    #maxTryNum = 3
    #for tries in range(maxTryNum):
    #    try:
    #        urlContext = urllib.request.urlopen(url).read()
    #    except:
    #        if tries < (maxTryNum - 1):
    #            continue
    #        else:
    #            print("Has tried %d times to access url %s, all failed!" % (maxTryNum, url))
    #            break
    urlContext = downloadFromURL(url)

    latestAmount = [0.00,0.00,0.00,0.00,0.00]

    if urlContext is None:
        return latestAmount

    #print(urlContext)
    strContext = urlContext.decode().strip()

    #details = (strContext[21:-22]).split(',","ya":["')
    details = (((strContext.split('"klines":["'))[1]).split('"]}});'))[0]
    #print(details)


#    totalAmount = 0
    if len(details) > 1:
        amountPerMin = details.split('","')

        #print(amountPerMin)
        #print(len(amountPerMin))

        if len(amountPerMin) > 0:
            # for i in range(len(amountPerMin)):
            #     #print(amountPerMin[i])
            #     if amountPerMin[i] == ",,,,":
            #         break
            #     else:
            #         latestAmount = amountPerMin[i].split(',')
            latestAmountDateTime = (amountPerMin[-1].split(','))[0:1]
            latestAmountOrg = (amountPerMin[-1].split(','))[1:]
    latestAmount = [float(amount) / 10000 for amount in latestAmountOrg]
    #print(latestAmount)
    return latestAmount
def getDataFromExcel(file_url, title=None):
    import xlrd
    import os
    import xlwt
    #from xlutils.copy import copy;
   # 首先在该目录下创建一个excel文件,用于存储所有excel文件的数据
    #file_name = root + '/' + file_name
    #create_excel(file_name, title)

    list_row_data = []
    #for f in allFile_url:
        # 打开excel文件
    #print('打开%s文件' % file_url)
    excel = xlrd.open_workbook(file_url)
    # 根据索引获取sheet，这里是获取第一个sheet
    table = excel.sheet_by_index(0)
    #print('该文件行数为：%d，列数为：%d' % (table.nrows,table.ncols))


    # 获取excel文件所有的行
    for i in range(table.nrows):
        # 跳过第零行，一般为表头
        if i == 0:
            i += 1
            continue
        row = table.row_values(i)  # 获取整行的值，返回列表
        print("Row Content:", i-1, row[0])
        if row[0] == '数据来源:通达信':
            break
        list_row_data.append(int(row[0]))

    #print('总数据量为%d' % len(list_row_data))
    #print('数据内容为%s' % list_row_data)
    return list_row_data
    # 写入all文件
    #add_row(list_row_data, file_name)
#print(getLatestAmountByurls([id for id in range(100)]))

"""
@xw.func
@xw.arg('times', np.array, ndim=1)
@xw.ret(index=False, header=False)
"""
#def getTotalStockAmountDetail(url):
def getTotalStockAmountDetail(times):

    #print(nowTime)
    #import urllib.request
    #url = "http://push2.eastmoney.com/api/qt/clist/get?pn=1&pz=" + str(times[0]) + "&po=1&np=1&ut=b2884a393a59ad64002292a3e90d46a5&fltt=2&invt=2&fid0=f4001&fid=f3&fs=m:0+t:6+f:!2,m:0+t:13+f:!2,m:0+t:80+f:!2,m:1+t:2+f:!2,m:1+t:23+f:!2&stat=1&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f14,f15,f16,f17,f18,f19,f20,f39,f62,f184,f66,f69,f72,f75,f78,f81,f84,f87,f204,f205,f124&rt=53075970&cb=jQuery183005756166574258548_1592279070569&_=1592279102840"
    url = (
    "http://push2.eastmoney.com/api/qt/clist/get?"
    "pn=1&pz=" + str(times[0]) +
    "&po=1&np=1&ut=b2884a393a59ad64002292a3e90d46a5"
    "&fltt=2&invt=2&fid0=f4001&fid=f3"
    "&fs=m:0+t:6+f:!2,m:0+t:13+f:!2,m:0+t:80+f:!2,m:1+t:2+f:!2,m:1+t:23+f:!2"    "&stat=1"
    "&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f14,f15,f16,"
    "f17,f18,f19,f20,f39,f62,f184,f66,f69,f72,f75,f78,f81,f84,f87,f204,f205,f124"
    "&rt=53075970&cb=jQuery183005756166574258548_1592279070569&_=1592279102840"
    )
    #print(url)
    #urlContext = urllib.request.urlopen(url).read()
    #urlContext = downloadFromURL("http://push2.eastmoney.com/api/qt/clist/get?pn=1&pz=" + str(times[0]) + "&po=1&np=1&ut=b2884a393a59ad64002292a3e90d46a5&fltt=2&invt=2&fid0=f4001&fid=f3&fs=m:0+t:6+f:!2,m:0+t:13+f:!2,m:0+t:80+f:!2,m:1+t:2+f:!2,m:1+t:23+f:!2&stat=1&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f14,f39,f62,f184,f66,f69,f72,f75,f78,f81,f84,f87,f204,f205,f124&rt=53075970&cb=jQuery183005756166574258548_1592279070569&_=1592279102840")
    urlContext = downloadFromURL(url)

    if urlContext is None:
        print("⚠️ 警告：urlContext 为空，跳过解码")
        totalStockAmountDetail = []
        return

    import json

    #data = json.load(urlContext)


    #print(data)
    #text = str(urlContext.decode('GBK'), encoding = "GBK")
    #text = urlContext.decode('GBK')
    #text = urlContext.decode('utf-8')


    if urlContext is not None:
        text = urlContext.decode('utf-8')
    else:
        print("urlContext is None")
        totalStockAmountDetail = []
        return

    # json_str = json.dumps(text)
    # data2 = json.loads(json_str)


    #yesPrice = float(((((text).split('var yZGhWyHM = {"data":['))[1]).split('","data":'))[0])
    #records = (((text.split('var yZGhWyHM = {"data":[')[1]).split('],"TotalCount":'))[0]
    records = (text.split('"diff":[{'))[1]
    records = (records.split('}]}});'))[0]
    records = (records.split('},{'))
    #records = data2['diff']
    #records = data2['jQuery183005756166574258548_1592279070569']['data']['diff']
    #print(records[-1])
    #print(records)
    #print(records)
    #json_str = json.dumps(records)
    #print(json_str)

    #json_dic = json.loads(json_str)
    #print(json_dic)
    #print(json_dic['diff'])
    totalStockAmountDetail = [0 for _ in range(len(records))]
    perStockAmountDetail = []


    import json
    for i in range(len(records)):
        #records[i] = ('"data":{"NOTICEDATE"' + records[i] + "}")
        #records[i] = ('{"NOTICEDATE"' + records[i] + "}")
        perStockAmountDetail = []
        #json_dic = json.loads(records[i])
        data = "{" + records[i] + "}"
        #json_str = json.dumps(data)
        json_dic = json.loads(data)
        #print(json_dic)
        #print(type(json_dic))

        stockID = json_dic["f12"]
        #stockMarket = "3" if ((('{:0>6}'.format(stockID))[0]) == "3") else ("4" if ((('{:0>6}'.format(stockID))[0:2]) == "68") else ("2" if ((('{:0>6}'.format(stockID))[0]) == "0") else "1")

        stockID_formatted = '{:0>6}'.format(stockID)

        if stockID_formatted[0] == "3":
            stockMarket = "3"
        elif stockID_formatted[0:2] == "68":
            stockMarket = "4"
        elif stockID_formatted[0] == "0":
            stockMarket = "2"
        else:
            stockMarket = "1"

        #print(stockMarket)

        stockName = json_dic["f14"]
        stockPrice = 0 if (json_dic["f2"] == "-") else json_dic["f2"]
        stockPercent = 0 if (json_dic["f3"] == "-") else json_dic["f3"]
        changesPercent = 0 if (json_dic["f8"] == "-") else json_dic["f8"]
        topPrice = 0 if (json_dic["f15"] == "-") else json_dic["f15"]
        lowPrice = 0 if (json_dic["f16"] == "-") else json_dic["f16"]
        openPrice = 0 if (json_dic["f17"] == "-") else json_dic["f17"]
        yesPrice = 0 if (json_dic["f18"] == "-") else json_dic["f18"]
        #print(changesPercent)
        liutong = 0 if (json_dic["f39"] == "-") else json_dic["f39"] / 10000
        NetBuyAmount = 0 if (json_dic["f62"] == "-") else json_dic["f62"] / 10000
        NetBuyPercent = 0 if (json_dic["f184"] == "-") else json_dic["f184"]
        GreatBuyAmount = 0 if (json_dic["f66"] == "-") else json_dic["f66"] / 10000
        GreatBuyPercent = 0 if (json_dic["f69"] == "-") else json_dic["f69"]
        LargeBuyAmount = 0 if (json_dic["f72"] == "-") else json_dic["f72"] / 10000
        LargeBuyPercent = 0 if (json_dic["f75"] == "-") else json_dic["f75"]
        MiddleBuyAmount = 0 if (json_dic["f78"] == "-") else json_dic["f78"] / 10000
        MiddleBuyPercent = 0 if (json_dic["f81"] == "-") else json_dic["f81"]
        SmallBuyAmount = 0 if (json_dic["f84"] == "-") else json_dic["f84"] / 10000
        SmallBuyPercent = 0 if (json_dic["f87"] == "-") else json_dic["f87"]


        #print(stockID, stockName, getLatestAmountByurls(stockID))
        #print(stockID, stockName, stockPrice, stockPercent, NetBuyAmount, NetBuyPercent, GreatBuyAmount, GreatBuyPercent, LargeBuyAmount, LargeBuyPercent, MiddleBuyAmount, MiddleBuyPercent, SmallBuyAmount, SmallBuyPercent)
        perStockAmountDetail = [stockMarket, stockID, stockName,  liutong,  stockPrice, stockPercent, changesPercent, topPrice, lowPrice,  openPrice,  yesPrice, NetBuyAmount, NetBuyPercent, GreatBuyAmount, GreatBuyPercent, LargeBuyAmount, LargeBuyPercent, MiddleBuyAmount, MiddleBuyPercent, SmallBuyAmount, SmallBuyPercent]

        totalStockAmountDetail[i] = perStockAmountDetail
        #print(totalStockAmountDetail)
        #print((json_dic['CDSY_SECUCODES'][0]['SECURITYCODE']))
        #print(type(json_dic['CDSY_SECUCODES'][0]))
    #print(records[0])
    #print(records[1])
    #print(records[-1])
    #print(totalStockAmountDetail)
    return totalStockAmountDetail

    #import json
    #json_str = json.dumps(text)
    #print ("Python 原始数据：", repr(text))
    #print ("JSON 对象：", json_str)

#getNoticeTopics("http://data.eastmoney.com/notices/getdata.ashx?StockCode=&FirstNodeType=0&CodeType=1&PageIndex=1&PageSize=2500&jsObj=yZGhWyHM&SecNodeType=0&Time=&rt=51105891")
#getIDByPercent("http://push2.eastmoney.com/api/qt/clist/get?pn=1&pz=5000&po=1&np=1&ut=b2884a393a59ad64002292a3e90d46a5&fltt=2&invt=2&fid0=f4001&fid=f3&fs=m:0+t:6+f:!2,m:0+t:13+f:!2,m:0+t:80+f:!2,m:1+t:2+f:!2,m:1+t:23+f:!2&stat=1&fields=f12,f14,f2,f3,f62,f184,f66,f69,f72,f75,f78,f81,f84,f87,f204,f205,f124&rt=53075970&cb=jQuery183005756166574258548_1592279070569&_=1592279102840")

In [224]:
import numpy as np
from IPython.display import Audio, display

def play_sound(duration=1, freq=440):
    """
    在Google Colab上自动播放提示音
    :param duration: 持续时间（秒）
    :param freq: 频率（Hz）
    """
    sample_rate = 44100  # 采样率
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)  # 时间轴
    audio = 0.5 * np.sin(2 * np.pi * freq * t)  # 正弦波生成音频
    display(Audio(audio, rate=sample_rate, autoplay=True))  # 自动播放音频

# 调用提示音函数
#play_sound()

#import pandas as pd
from datetime import datetime
#from google.colab import drive

# 挂载 Google 云端硬盘
#drive.mount('/content/drive')

def log_dataframe_to_drive(df, drive_folder_path="/content/drive/My Drive/logs/"):
    """
    将 DataFrame 写入当天的日志文件并保存在 Google 云端硬盘中。

    参数:
    df (pd.DataFrame): 需要写入日志的 DataFrame。
    drive_folder_path (str): 云端硬盘中日志文件保存的目录路径。
    """
    from os import makedirs, path

    # 获取当天日期
    today = datetime.now().strftime("%Y-%m-%d")
    log_filename = f"{drive_folder_path}stock_log_{today}.log"

    # 确保日志目录存在
    if not path.exists(drive_folder_path):
        makedirs(drive_folder_path)

    # 写入日志文件
    with open(log_filename, "a") as log_file:  # "a" 模式为追加模式
        log_file.write(f"Log Date: {today}\n")
        log_file.write(df.to_string(index=False))  # 写入 DataFrame 内容
        log_file.write("\n\n")  # 换行分隔
    print(f"日志文件已写入: {log_filename}")

import sys
from datetime import datetime
#from google.colab import drive


def setup_realtime_logging(log_folder="/content/drive/My Drive/logs/"):
    """
    设置实时日志记录，将所有控制台输出同时写入日志文件。

    参数:
    log_folder (str): 云端硬盘中日志文件保存的目录路径。
    """
    import os

    # 挂载 Google 云端硬盘
    #drive.mount('/content/drive')


    # 获取当天日期
    today = datetime.now().strftime("%Y-%m-%d")
    log_filename = os.path.join(log_folder, f"realtime_log_{today}.log")

    # 确保日志目录存在
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)

    # 定义写入文件和控制台的流
    class Tee:
        def __init__(self, file, stream):
            self.file = file
            self.stream = stream

In [225]:
import time
from datetime import datetime, timedelta

# 示例：股票交易的开盘和收盘时间（9:15-15:00）
MARKET_OPEN = "09:15:00"
MARKET_CLOSE = "15:30:00"


# 获取当前 UTC 时间
#utc_time = datetime.utcnow()

# 加上 8 小时时差
#china_time = utc_time + timedelta(hours=8)
#print("当前中国北京时间:", china_time.strftime('%Y-%m-%d %H:%M:%S'))

#from datetime import datetime
import pytz

# 设置时区为中国北京时间
china_timezone = pytz.timezone('Asia/Shanghai')

# 获取当前中国北京时间
#current_time = datetime.now(china_timezone)
#print("当前中国北京时间:", current_time.strftime('%Y-%m-%d %H:%M:%S'))

def is_trading_day():
    """判断是否为交易日（这里只是简单示例，实际需通过API或日历实现）"""

    # 假设周一到周五为交易日
    today = datetime.now(china_timezone).weekday()
    #print(today)
    return today < 5  # 0=Monday, ..., 4=Friday

def is_trading_time():
    """判断当前时间是否在交易时间内"""
    now = datetime.now(china_timezone)
    current_time = now.strftime("%H:%M:%S")
    return MARKET_OPEN <= current_time <= MARKET_CLOSE

def wait_until_next_open():
    """等待下一个交易日的开盘时间"""
    now = datetime.now(china_timezone)
    # 如果今天是交易日且尚未开盘，等待今天开盘
    if is_trading_day() and now.strftime("%H:%M:%S") < MARKET_OPEN:
        next_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
    else:
        # 否则等待下一个交易日的开盘时间
        next_open = now + timedelta(days=(1 if now.weekday() < 4 else 7 - now.weekday()))
        next_open = next_open.replace(hour=9, minute=15, second=0, microsecond=0)

    sleep_seconds = (next_open - now).total_seconds()
    print(f"等待到下一个交易日开盘时间：{next_open}")
    time.sleep(sleep_seconds)

In [226]:
#install_packages(["gTTS", "pypinyin"])

from gtts import gTTS
from IPython.display import Audio, display
import os
import pypinyin  # 用于拼音转换

def generate_and_play_stock_sounds(stock_data):
    """
    针对多个股票生成详细语音并依次播放。
    :param stock_data: 股票数据列表，例如 [{"code": "000001", "name": "平安银行", "price": 12.34, "change": "+3.2%"}]
    """
    # 临时文件存储目录
    temp_dir = "stock_sounds"
    os.makedirs(temp_dir, exist_ok=True)

    # 遍历股票数据生成语音
    for stock in stock_data:
        # 拼音转换
        #code_pinyin = " ".join(pypinyin.lazy_pinyin(stock['stockID']))
        name_pinyin = " ".join(pypinyin.lazy_pinyin(stock['stockName']))

        # 构造语音播报内容，拼音化股票代码
        #text = (
        #    f"股票代码 {code_pinyin}，"
        text = (
            f"股票代码 {str(stock['stockID'])}，"
            f"名称 {name_pinyin}，"
            f"当前价格 {stock['stockPrice']} 元，"
            f"涨跌幅 {stock['changesPercent']}。"
        )

        # 使用 gTTS 生成语音
        tts = gTTS(text=text, lang="zh-cn")

        # 保存为临时文件
        temp_file = os.path.join(temp_dir, f"{stock['code']}.mp3")
        tts.save(temp_file)

        # 播放语音
        print(f"正在播报: {text}")
        display(Audio(temp_file, autoplay=True))

In [227]:

# 示例股票数据
#stock_data = [
#    {"code": "000001", "name": "平安银行", "price": 12.34, "change": "+3.2%"},
#    {"code": "600519", "name": "贵州茅台", "price": 1952.76, "change": "+1.5%"},
#    {"code": "300750", "name": "宁德时代", "price": 425.68, "change": "-0.8%"}
#]
from gtts import gTTS
from IPython.display import Audio, display
import os
import pypinyin  # 用于拼音转换
import time

def generate_and_play_stock_sounds_from_df(stock_df):
    """
    针对多个股票生成详细语音并依次播放，输入为 DataFrame 类型的数据。
    :param stock_df: 股票数据 DataFrame，例如包含 'stockID', 'stockName', 'price', 'change' 等列
    """

   # 临时文件存储目录
    temp_dir = "stock_sounds"
    os.makedirs(temp_dir, exist_ok=True)

    # 遍历股票数据生成语音
    for index, row in stock_df.iterrows():
        # 拼音转换

        # 数字到中文的映射
        chinese_mapping = {
        '0': '零', '1': '一', '2': '二', '3': '三', '4': '四',
        '5': '五', '6': '六', '7': '七', '8': '八', '9': '九'
        }

        # 将股票代码中的每个数字转换为中文
        chinese_code = ''.join(chinese_mapping[digit] for digit in str(row['stockID']))




        #code_pinyin = " ".join(pypinyin.lazy_pinyin(row['stockID']))
        name_pinyin = " ".join(pypinyin.lazy_pinyin(row['stockName']))

        # 构造语音播报内容，拼音化股票代码
        text = (
            f"股票代码 {chinese_code}, "
            f"名称 {name_pinyin}，"
            f"当前价格 {row['stockPrice']} 元，"
            f"涨跌幅 {row['stockPercent']}。"
        )

        # 使用 gTTS 生成语音
        tts = gTTS(text=text, lang="zh-cn")

        # 保存为临时文件
        temp_file = os.path.join(temp_dir, f"{row['stockID']}.mp3")
        time.sleep(1)
        tts.save(temp_file)

        # 播放语音
        #print(f"正在播报: {text}")
        display(HTML(f'<span style="font-weight:bold; color:#FF0000;">{text}</span>'))

        display(Audio(temp_file, autoplay=True))

# 调用函数，传入 DataFrame
#generate_and_play_stock_sounds_from_df(stock_data)
# 调用函数
#generate_and_play_stock_sounds(stock_data)

In [228]:
import pandas as pd

pd.set_option('display.max_columns', None, 'display.max_rows', None)

#if __name__ == '__main__':
def OneTimeRunJob(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist):
    global STOCK_TRACKER
    #STOCK_TRACKER = {}

    MARKET_OPEN_25 = "09:25:00"

    #print(getFirstAmountByurls([2824.00,600230.00,300939.00]))
    #print(getLatestAmountByurls([2824.00,600230.00,300939.00]))
    topStocks = getTotalStockAmountDetail([stockNum])

    #print(stockNum)


    perStockAmountColumns = ["stockMarket", "stockID", "stockName",  "liutong",   "stockPrice", "stockPercent",  "changesPercent", "topPrice",  "lowPrice", "openPrice", "yesPrice",  "NetBuyAmount", "NetBuyPercent", "GreatBuyAmount", "GreatBuyPercent", "LargeBuyAmount", "LargeBuyPercent", "MiddleBuyAmount", "MiddleBuyPercent", "SmallBuyAmount", "SmallBuyPercent"]

    df = pd.DataFrame(topStocks, columns=perStockAmountColumns)
    df["openPercent"] = (df["openPrice"] - df["yesPrice"])/ df["yesPrice"] * 100  #

    #print(getFirstAmountByurls([id for id in range(100)]))
    #print(type(topStocks))
    #print(df)

    now = datetime.now(china_timezone)
    current_time = now.strftime("%H:%M:%S")

    filtered_df1 = df[(df["stockMarket"] < '4') &
                   ((df["stockName"].str.startswith("N")) |
                   (df["stockName"].str.startswith("C")) |
                   (df["stockMarket"].isin(stockmarketlist)) |
                   (df["NetBuyPercent"].astype(int) >= 35) |
                   (df["GreatBuyPercent"].astype(int) >= 35) |
                   (df["changesPercent"].astype(float) >= 4.00))]
    """
    filtered_df1 = df[(df["stockMarket"] < '4') &
                   ((df["stockName"].str.startswith("N")) |
                   (df["stockName"].str.startswith("C")) |
                   (df["stockMarket"] == '3') |
                   (df["NetBuyPercent"] >= 35) |
                   (df["GreatBuyPercent"] >= 35) |
                   (df["changesPercent"] >= 4.00))]
    """
    #print(filtered_df1)
    #if "09:25:00" <= current_time and "09:40:00" > current_time:
        #print("1")
        #NetBuyPercent = df[(df["stockMarket"] != '4') & (df["openPercent"] >= 10.00)]
        #filtered_df3 = df[((df["stockName"].str.startswith("N")) |
        #           (df["stockName"].str.startswith("C")) |
        #           (df["stockMarket"] != '4')) &
        #           (df["openPercent"] >= 10.00)]
        #filtered_df3 = filtered_df1[(filtered_df1["openPercent"] >= 10.00)]

    #elif "09:40:00" <= current_time and "09:45:00" > current_time:
        #print("2")

    #    #filtered_df3 = df[(df["stockMarket"] != '4') & (df["openPercent"] >= 7.00)]
    #    filtered_df3 = df[((df["stockName"].str.startswith("N")) |
    #               (df["stockName"].str.startswith("C")) |
    #               (df["stockMarket"] != '4')) &
    #               (df["openPercent"] >= 7.00)]
        #filtered_df3 = filtered_df1[(filtered_df1["openPercent"] >= 7.00)]
    #else:
        #print("3")

        #filtered_df3 = df[(df["stockMarket"] != '4') &  ((df["openPercent"] >= 4.5) | ((df["openPercent"] >= 4.5) &  (df["NetBuyAmount"] >= 10000)))]
        #filtered_df3 = df[((df["stockName"].str.startswith("N")) |
        #           (df["stockName"].str.startswith("C")) |
        #           (df["stockMarket"] != '4')) &
        #           ((df["openPercent"] >= 4.5) | ((df["openPercent"] >= monitorLowPercent) &  (df["NetBuyAmount"] >= 10000)))]
    filtered_df3 = filtered_df1[((filtered_df1["openPercent"].astype(float) >= 4.5) |
                                     ((filtered_df1["openPercent"].astype(float) >= monitorLowPercent) &
                                      (filtered_df1["NetBuyAmount"].astype(float) >= monitorBuyAmount)))]


    #filtered_df3 = df[(df["stockMarket"] != '4') &  (df["openPercent"] >= 4.5)]
    #filtered_df3 = df[(df["stockMarket"] != '4') &  ((df["openPercent"] >= 4.5) | ((df["openPercent"] >= 3.5) &  (df["NetBuyAmount"] >= 10000)))]
    #filtered_df3 = df[(df["stockMarket"] == '3') | (df["stockName"].str.startswith("N"))]
    # 重设索引
    df3 = filtered_df3.reset_index(drop=True)
    # 将 DataFrame 转换为字典，orient='index' 会将每一行转换为字典
    #temp_dict = df3.to_dict(orient='index')
    #print(df3)



    #processSTOCK_TRACKER_firstTime()
    #print(STOCK_TRACKER)
    #processSTOCK_TRACKER_daily
    #update_latest_data(df3)
    #print(df["stockID"])
    #firstA = getFirstAmountByurls(getDataFromExcel(r'F:\new_dfzq_v6\T0002\export\沪深Ａ股20190401.xls'))
    firstAmount = getFirstAmountByurls(df3["stockID"], df3["liutong"]).tolist()
    firstAmountColumns = ["PanQianChanceRadio", "PanQianVolume",   "changeRadioTime5",  "changeRadioTime40",   "changeRadioTime60",   "changeRadioTime70",    "ZTPrice",     "totalAmount", "PanQianBelowYes", "PanQianZT", "PanQianZTSec", "PanQianLowPerc", "PanQianNowPerc", "totalAmount2", "totalAmount3", "bigOrderNumber", "bigOrderNumberPerc", "PanQianUpFlag"]
    #print(type(firstAmount))
    firstA = pd.DataFrame(firstAmount, columns=firstAmountColumns)
    #print(firstA)
    # 合并
    result = pd.concat([df3, firstA], axis=1)
    result["PanQianChanceRadio"] = result["PanQianVolume"] / result["liutong"]  #
    #print(results["PanQianChanceRadio"])     #
    # firstA = pd.DataFrame(firstAmount, columns=firstAmountColumns)
    #for i in range(len(firstA)):
    #print(firstAmount)
    #print(firstA["totalAmount"])
    #goodStocks = result[(result["PanQianUpFlag"] == "Y") & ((result["PanQianChanceRadio"] >= 5.00) | (result["changeRadioTime5"] < "093000")  |  (result["changeRadioTime40"] <= "094000") | (result["changeRadioTime60"] <= "100000") | (result["changeRadioTime70"] <= "113000") | (result["PanQianZT"] == "Y") & (result["PanQianZTSec"] >= 300) & (result["PanQianLowPerc"] >= np.where(result["stockMarket"] == "3", 4.50, 3.00)) & (result["PanQianNowPerc"] >= np.where(result["stockMarket"] == "3", 10.00, 5.00)) & (result["PanQianNowPerc"] - result["PanQianLowPerc"] <= 3))]
    #print(goodStocks)
    #print(result)     #
    print("高开股|机构净买额巨大股：")
    print(result)
    #df4 = result[
    #(result["PanQianChanceRadio"] >= 4.0) |
    #((~result["stockName"].str.startswith("N")) & (result["openPercent"] >= 4.5) & (result["PanQianLowPerc"] >= 2.5))
    #]
    now = datetime.now(china_timezone)
    current_time = now.strftime("%H:%M:%S")

    #df4 = result[
    #    (result["PanQianChanceRadio"] >= 4.0) |
    #    ((result["stockMarket"].isin(stockmarketlist)) & (~result["stockName"].str.startswith("N")) & (result["openPercent"] >= monitorLowPercent)) #& (result["PanQianLowPerc"] >= 0.00))
    #    ]
    #df4 = result[
    #    (result["PanQianChanceRadio"] >= 4.0) |
    #    ((result["stockMarket"].isin(stockmarketlist)) & (result["openPercent"] >= monitorLowPercent)) #& (result["PanQianLowPerc"] >= 0.00))
    #    ]
    #filtered_df4 = result[((result["PanQianUpFlag"] == "Y") &
    #                       (result["PanQianLowPerc"] >= 0.00))]
    #                       ((result["openPercent"] - result["PanQianLowPerc"]) <= 10))]
    filtered_df4 = result[(result["PanQianLowPerc"].astype(float) >= 0.00) |
                          (result["stockName"].str.startswith("N")) |
                          (result["stockName"].str.startswith("C"))]

    #print("高开股|机构净买额巨大股&高开&盘前低振幅股：")
    #print(filtered_df4)
    df4 = filtered_df4[
     (filtered_df4["PanQianChanceRadio"].astype(float) >= 4.0) |
     (((filtered_df4["stockMarket"].isin(stockmarketlist)) |
       (filtered_df4["stockName"].str.startswith("N")) |
       (filtered_df4["stockName"].str.startswith("C"))) &
      (filtered_df4["openPercent"].astype(float) >= monitorLowPercent)) #& (filtered_df4["PanQianLowPerc"].astype(float) >= 0.00))
     ]
    #if "09:40:00" > current_time:
        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) |
        #((~result["stockName"].str.startswith("N")) & (result["PanQianZTSec"] > 9) & (result["openPercent"] >= 1.5) & (result["PanQianLowPerc"] >= 0.0))
        #]
        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) | (result["NetBuyPercent"] >= 35) |
        #((result["stockMarket"].isin(stockmarketlist)) & (~result["stockName"].str.startswith("N")) & (result["PanQianZTSec"] > 9) & (result["openPercent"] >= monitorLowPercent) & (result["PanQianLowPerc"] >= 0.0))
        #]
        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) | (result["NetBuyPercent"] >= 35) |
        #((result["stockMarket"] == '3') & (~result["stockName"].str.startswith("N")) & (result["PanQianZTSec"] > 9) & (result["openPercent"] >= monitorLowPercent) & (result["PanQianLowPerc"] >= 0.0))
        #]
        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) |
        #((result["stockMarket"].isin(stockmarketlist)) & (~result["stockName"].str.startswith("N")) & (result["PanQianZTSec"] > 9) & (result["openPercent"] >= monitorLowPercent) & (result["PanQianLowPerc"] >= 0.0))
        #]
    #else:
        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) |
        #((~result["stockName"].str.startswith("N")) & (result["openPercent"] >= monitorLowPercent) & (result["PanQianLowPerc"] >= monitorLowPercent       #]

        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) |
        #((result["stockMarket"].isin(stockmarketlist)) & (~result["stockName"].str.startswith("N")) & (result["openPercent"] >= monitorLowPercent)) #& (result["PanQianLowPerc"] >= 0.00))
        #]

        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) | (result["NetBuyPercent"] >= 35) |
        #((result["stockMarket"].isin(stockmarketlist)) & (~result["stockName"].str.startswith("N")) & (result["openPercent"] >= monitorLowPercent) & (result["PanQianLowPerc"] >= 0.00))
        #]

        #df4 = result[
        #(result["PanQianChanceRadio"] >= 4.0) | (result["NetBuyPercent"] >= 35) |
        #((result["stockMarket"] == '3') & (~result["stockName"].str.startswith("N")) & (result["openPercent"] >= monitorLowPercent) & (result["PanQianLowPerc"] >= 0.00))
        #]

    #df4 = result[
    #(result["PanQianChanceRadio"] >= 4.0) |
    #((~result["stockName"].str.startswith("N")) & (result["openPercent"] >= 1.5) & (result["PanQianLowPerc"] >= 0.0))
    #]
    #设置 'col2' 为索引
    #temp_dict = df.set_index('col2', inplace=True)
    print("今天关注股：")
    print(df4)

    # 选择 'col1'、'col3'、'col4' 和 'col5' 作为值列
    value_columns = ['stockID', "stockMarket", "stockName", "stockPrice", 'yesPrice', 'ZTPrice', 'openPercent', 'PanQianChanceRadio', 'PanQianUpFlag', 'PanQianZTSec', 'NetBuyPercent', 'NetBuyAmount', 'liutong']

    #转换为字典
    STOCK_TRACKER = df4[value_columns].set_index('stockID').T.to_dict('dict')

    # 构建最终的字典，以 stockID 的值为键，包含所有字段的字典为值
    #STOCK_TRACKER = {row['stockID']: row for row in temp_dict.values()}
    new_data = {
                "baseline_low": None,  # 基准最低价
                "baseline_high": None, # 基准最高价                "time": 0,
                "high": 0.00,
                "low": 0.00,
                "base_high_updates": 0,
                "high_updates": 0,
                "base_low_updates": 0,
                "low_updates": 0,
                "buytime": "",  #二次涨开始时间
                "buyprice": 0.00,  #二次涨开始价格
            }


    # 遍历原始字典，为每个值添加新的字典
    for key, value in STOCK_TRACKER.items():
        value.update(new_data)
    # 输出结果
    #print(STOCK_TRACKER)


    """判断当前时间是否在交易时间内"""
    now = datetime.now(china_timezone)
    current_time = now.strftime("%H:%M:%S")
    #if MARKET_OPEN_25 >= current_time:
    #    print(df3)
    #    print(result)
    # 修正括号和逻辑
    goodStocks = result[
    (result["PanQianUpFlag"] == "Y") &
    (
        (result["PanQianChanceRadio"] >= 5.00) |
        (result["changeRadioTime5"] < "093000") |
        (result["changeRadioTime40"] <= "094000") |
        (result["changeRadioTime60"] <= "100000") |
        (result["changeRadioTime70"] <= "113000") |
        ((result["NetBuyPercent"] >= 40) & (result["PanQianLowPerc"] >= 3.5)) |
        (
            (result["PanQianZT"] == "Y") &
            (result["PanQianZTSec"] >= 450) &
            (result["PanQianLowPerc"] >= np.where(result["stockMarket"] == "3", 4.50, 3.00)) &
            (result["PanQianNowPerc"] >= np.where(result["stockMarket"] == "3", 10.00, 5.00)) &
            ((result["PanQianNowPerc"] - result["PanQianLowPerc"]) <= 3)
        )
    )
    ]
    print(goodStocks)
    buyStocks = goodStocks[(((goodStocks["stockPrice"] < goodStocks["ZTPrice"]) & (~goodStocks["stockName"].str.contains("ST"))) |  (goodStocks["stockName"].str.startswith("N"))) & ( goodStocks["NetBuyAmount"] >= 8000)]
    print(buyStocks)
    if not buyStocks.empty:
        print("有符合条件的股票")
        generate_and_play_stock_sounds_from_df(buyStocks)

        #play_sound()
        print(buyStocks)

    else:
        print("都涨停了")

    # 调用函数写入日志
    #log_dataframe_to_drive(result)
    #log_dataframe_to_drive(goodStocks)
    #log_dataframe_to_drive(buyStocks)
    #return df3

In [229]:

import pandas as pd
import numpy as np
import time
from datetime import datetime

def run_task(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist):
    """重复运行的任务"""
    print(f"执行任务，当前时间：{datetime.now()}")
    OneTimeRunJob(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist)

def process_tracker_and_print():
    """处理并打印 STOCK_TRACKER 的内容"""
    global STOCK_TRACKER

    # 每日更新处理
    processSTOCK_TRACKER_daily()

    # 将 STOCK_TRACKER 转为 DataFrame
    df_tracker = pd.DataFrame.from_dict(STOCK_TRACKER, orient="index")
    print(df_tracker)

    # 判断是否存在关键列
    if ("buytime" in df_tracker.columns) and ("buyFlag" in df_tracker.columns):
        print("🍎🍎🍎赶快设置以下股票的条件买单吧！！！")

        # 替换无穷大和空值
        df_tracker = df_tracker.replace([np.inf, -np.inf], 0).fillna(0)

        # 选出符合条件的股票
        good_tracker = df_tracker[
            (
                (
                    (df_tracker["high_updates"].astype(int) > 1) |
                    (df_tracker["NetBuyAmount"].astype(float) > 5000)
                )
                &
                (
                    (df_tracker["buytime"] != "") |
                    (df_tracker["buyFlag"] != "")
                )
                & (
                    (df_tracker["PanQianZTSec"].astype(int) > 1) |
                    ((df_tracker["total_days"].astype(int) > 0) & (df_tracker["total_days"].astype(int) < 61)) |
                    (df_tracker["ZT_days"].astype(int) > 0)
                )
                & (
                    (df_tracker["rank"].astype(int) <= 100) |
                    ((df_tracker["total_days"].astype(int) > 0) & (df_tracker["total_days"].astype(int) < 61))
                )
            )
            |
            (
                (df_tracker["openPercent"].astype(float) > 19) &
                (df_tracker["NetBuyAmount"].astype(float) > 0) &
                (df_tracker["high"] == df_tracker["low"])
            )
        ]

        print(good_tracker)
    else:
        print("列 'buytime' 不存在，检查 STOCK_TRACKER 数据格式")
def main(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist):
    global STOCK_TRACKER
    """主程序"""
    STOCK_TRACKER = {}

    run_task(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist)
    process_tracker_and_print()

    while True:
        if is_trading_day() and is_trading_time():
            run_task(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist)
            print("************************")
            process_tracker_and_print()
            time.sleep(5)  # 改为 60 更合理
        else:
            wait_until_next_open()

In [230]:


if __name__ == "__main__":

    # 假设 STOCK_TRACKER 是全局字典
    STOCK_TRACKER = {}

    stockNum = 500
    #stockNum = 200
    monitorLowPercent = 1.50
    #monitorLowPercent = 1.00
    #monitorBuyAmount = 1000
    monitorBuyAmount = 3000
    #stockmarketlist = ['3', '2', '1']
    stockmarketlist = ['3']

    # 调用函数设置日志
    #setup_realtime_logging()
    #play_sound()
    main(stockNum, monitorLowPercent, monitorBuyAmount, stockmarketlist)

执行任务，当前时间：2025-06-12 05:47:51.852730
At Thu Jun 12 05:47:53 2025 on EastMoney...
all DONE at: Thu Jun 12 05:47:59 2025
高开股|机构净买额巨大股：
   stockMarket stockID stockName      liutong  stockPrice  stockPercent  \
0            1  603382       N海阳    3552.0860       60.97        430.17   
1            3  300237      美晨科技  144190.1430        2.50         20.19   
2            3  300949      奥雅股份    3430.1063       52.80         20.00   
3            1  601279      英利汽车  158578.5985        4.36         10.10   
4            1  603335       迪生力   42814.4600        5.91         10.06   
5            2  002181     粤 传 媒  113452.0181        8.88         10.04   
6            2  002615       哈尔斯   30604.8207        9.32         10.04   
7            1  600980      北矿科技   17567.7206       25.03         10.02   
8            1  603139      康惠制药    9988.0000       27.45         10.02   
9            1  603758      秦安股份   43879.7049       14.18         10.01   
10           1  603232      格尔软件   23130.0

UnboundLocalError: cannot access local variable 'filtered_df4' where it is not associated with a value